# Description

Python code to scrape player data from baseball-reference.com and rate players using SherCo PLUS ratings. Ratings are effective for any season since and including 1950.

# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import requests
from lxml import html
from bs4 import BeautifulSoup
import re
from urllib.parse import urlparse, parse_qs

from selenium import webdriver
import time

import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from openpyxl.utils import get_column_letter
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting import Rule
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.worksheet.dimensions import ColumnDimension, DimensionHolder

from bisect import bisect

import os

# Scrape season data

***NOTE: SIMPLY CHANGE THE YEAR VALUE THEN RUN ALL CELLS BELOW. THE RESULT WILL BE A FORMATTED EXCEL FILE WITH THAT YEAR'S PLAYER RATINGS. COMMENT OUT THE CLEAN UP SECTION IF YOU DON'T WANT TO REMOVE INTERIM FILES***

In [246]:
year = '1977'

In [247]:
url_season = 'https://www.baseball-reference.com/leagues/MLB/' + year + '-standings.shtml'

url_bat = 'https://www.baseball-reference.com/leagues/MLB/' + year + '-standard-batting.shtml'
url_pit = 'https://www.baseball-reference.com/leagues/MLB/' + year + '-standard-pitching.shtml'
url_fld = 'https://www.baseball-reference.com/leagues/MLB/' + year + '-standard-fielding.shtml'
url_cat = 'https://www.baseball-reference.com/leagues/MLB/' + year + '-specialpos_c-fielding.shtml'

In [248]:
url_cat

'https://www.baseball-reference.com/leagues/MLB/1977-specialpos_c-fielding.shtml'

In [249]:
session_requests = requests.session()

result = session_requests.get(url_season, headers = dict(referer = url_season))
result.status_code

200

In [250]:
# https://github.com/BenKite/baseball_data/blob/master/baseballReferenceScrape.py
def findTables(url):
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    divs = soup.find_all('div', id = "content")
    divs = divs[0].find_all("div", id=re.compile("^all"))
    ids = []
    for div in divs:
        searchme = str(div.find_all("table"))
        x = searchme[searchme.find("id=") + 3: searchme.find(">")]
        x = x.replace("\"", "")
        if len(x) > 0:
            ids.append(x)
    return(ids)

In [251]:
findTables(url_season)

['postseason',
 'standings_E',
 'standings_E',
 'standings_W',
 'standings_E',
 'standings_E',
 'standings_W',
 'expanded_standings_overall']

In [252]:
def pullTable(url, tableID):
    res = requests.get(url)
    ## Work around comments
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    tables = soup.find_all('table', id = tableID)
    data_rows = tables[0].find_all('tr')
    data_header = tables[0].find_all('thead')
    data_header = data_header[0].find_all("tr")
    data_header = data_header[0].find_all("th")
    game_data = [[td.getText() for td in data_rows[i].find_all(['th','td'])]
        for i in range(len(data_rows))
        ]
    data = pd.DataFrame(game_data)
    header = []
    for i in range(len(data.columns)):
        header.append(data_header[i].getText())
    data.columns = header
    data = data.loc[data[header[0]] != header[0]]
    data = data.reset_index(drop = True)
    return(data)

In [253]:
season = pullTable(url_season, 'expanded_standings_overall')
season.head()

,Rk,Tm,Lg,G,W,L,W-L%,R,RA,Rdiff,SOS,SRS,pythWL,Luck,vEast,vWest,Home,Road,ExInn,1Run,vRHP,vLHP,≥.500,<.500
0,1,KCR,AL,162,102,60,.630,5.1,4.0,1.1,-0.1,1.0,98-64,4,48-24,54-36,55-26,47-34,9-6,31-13,72-47,30-13,42-34,60-26
1,2,PHI,NL,162,101,61,.624,5.2,4.1,1.1,-0.1,1.0,98-64,3,55-35,46-26,60-21,41-40,9-7,22-23,77-45,24-16,49-41,52-20
2,3,NYY,AL,162,100,62,.617,5.1,4.0,1.1,-0.1,1.0,99-63,1,51-39,49-23,55-26,45-36,6-4,24-19,51-35,49-27,41-29,59-33
3,4,LAD,NL,162,98,64,.605,4.7,3.6,1.2,-0.1,1.0,101-61,-3,42-30,56-34,51-30,47-34,9-11,32-31,70-48,28-16,44-40,54-24
4,5,BAL,AL,161,97,64,.603,4.5,4.1,0.4,0.0,0.4,88-73,9,58-31,39-33,54-27,43-37,15-8,32-19,72-45,25-19,33-37,64-27


In [254]:
season["Tm"] = [re.sub('w‑', '', x) for x in season["Tm"]]
season["Tm"] = [re.sub('y‑', '', x) for x in season["Tm"]]

In [255]:
season

,Rk,Tm,Lg,G,W,L,W-L%,R,RA,Rdiff,SOS,SRS,pythWL,Luck,vEast,vWest,Home,Road,ExInn,1Run,vRHP,vLHP,≥.500,<.500
0,1,KCR,AL,162,102,60,.630,5.1,4.0,1.1,-0.1,1.0,98-64,4,48-24,54-36,55-26,47-34,9-6,31-13,72-47,30-13,42-34,60-26
1,2,PHI,NL,162,101,61,.624,5.2,4.1,1.1,-0.1,1.0,98-64,3,55-35,46-26,60-21,41-40,9-7,22-23,77-45,24-16,49-41,52-20
2,3,NYY,AL,162,100,62,.617,5.1,4.0,1.1,-0.1,1.0,99-63,1,51-39,49-23,55-26,45-36,6-4,24-19,51-35,49-27,41-29,59-33
3,4,LAD,NL,162,98,64,.605,4.7,3.6,1.2,-0.1,1.0,101-61,-3,42-30,56-34,51-30,47-34,9-11,32-31,70-48,28-16,44-40,54-24
4,5,BAL,AL,161,97,64,.603,4.5,4.1,0.4,0.0,0.4,88-73,9,58-31,39-33,54-27,43-37,15-8,32-19,72-45,25-19,33-37,64-27
5,6,BOS,AL,161,97,64,.603,5.3,4.4,0.9,-0.1,0.8,94-67,3,54-35,43-29,51-29,46-35,6-2,24-21,78-50,19-14,34-35,63-29
6,7,PIT,NL,162,96,66,.593,4.5,4.1,0.4,0.0,0.4,88-74,8,55-35,41-31,58-23,38-43,15-8,31-20,63-41,33-25,50-40,46-26
7,8,TEX,AL,162,94,68,.580,4.7,4.1,0.7,-0.1,0.6,92-70,2,42-30,52-38,44-37,50-31,14-6,30-29,70-47,24-21,36-39,58-29
8,9,CHW,AL,162,90,72,.556,5.2,4.8,0.5,0.0,0.4,88-74,2,39-33,51-39,48-33,42-39,6-5,19-24,61-51,29-21,39-36,51-36
9,10,CIN,NL,162,88,74,.543,5.0,4.5,0.5,-0.1,0.4,88-74,0,38-34,50-40,48-33,40-41,4-5,24-23,52-48,36-26,36-48,52-26


In [256]:
my_teams = list(season["Tm"])
my_teams.remove("Avg")
my_teams

['KCR',
 'PHI',
 'NYY',
 'LAD',
 'BAL',
 'BOS',
 'PIT',
 'TEX',
 'CHW',
 'CIN',
 'MIN',
 'STL',
 'HOU',
 'CHC',
 'SFG',
 'MON',
 'DET',
 'CAL',
 'CLE',
 'SDP',
 'MIL',
 'NYM',
 'SEA',
 'OAK',
 'ATL',
 'TOR']

In [257]:
team_url = 'https://www.baseball-reference.com/teams/NYY/' + year + '.shtml'

In [258]:
findTables(team_url)

['team_batting',
 'team_pitching',
 'appearances',
 'coaches',
 'standard_fielding',
 'players_value_batting',
 'players_value_pitching']

In [259]:
appearances = pullTable(team_url, 'appearances')
appearances.head()

,Name,Age,,B,T,Ht,Wt,DoB,Yrs,G,GS,Batting,Defense,P,C,1B,2B,3B,SS,LF,CF,RF,OF,DH,PH,PR,WAR,Salary,
0,Dell Alston,24,us US,L,R,"6' 0""",180,"Sep 22, 1952",1st,22,10,22,2,0,0,0,0,0,0,0,0,2,2,10,8,4,0.2,,
1,Dave Bergman,24,us US,L,L,"6' 1""",185,"Jun 6, 1953",2,5,0,5,5,0,0,2,0,0,0,1,1,1,3,0,1,1,-0.1,,
2,Paul Blair,33,us US,R,R,"6' 0""",168,"Feb 1, 1944",14,83,37,83,80,0,0,0,0,0,0,6,42,34,80,3,4,4,0.5,"$100,000",
3,Chris Chambliss,28,us US,L,R,"6' 1""",195,"Dec 26, 1948",7,157,150,157,157,0,0,157,0,0,0,0,0,0,0,0,4,0,2.4,"$240,000",
4,Ken Clay,23,us US,R,R,"6' 3""",185,"Apr 6, 1954",1st,21,3,0,21,21,0,0,0,0,0,0,0,0,0,0,0,0,-0.4,,


In [260]:
appearances["Tm"] = "NYY"
appearances.head()

,Name,Age,,B,T,Ht,Wt,DoB,Yrs,G,GS,Batting,Defense,P,C,1B,2B,3B,SS,LF,CF,RF,OF,DH,PH,PR,WAR,Salary,,Tm
0,Dell Alston,24,us US,L,R,"6' 0""",180,"Sep 22, 1952",1st,22,10,22,2,0,0,0,0,0,0,0,0,2,2,10,8,4,0.2,,,NYY
1,Dave Bergman,24,us US,L,L,"6' 1""",185,"Jun 6, 1953",2,5,0,5,5,0,0,2,0,0,0,1,1,1,3,0,1,1,-0.1,,,NYY
2,Paul Blair,33,us US,R,R,"6' 0""",168,"Feb 1, 1944",14,83,37,83,80,0,0,0,0,0,0,6,42,34,80,3,4,4,0.5,"$100,000",,NYY
3,Chris Chambliss,28,us US,L,R,"6' 1""",195,"Dec 26, 1948",7,157,150,157,157,0,0,157,0,0,0,0,0,0,0,0,4,0,2.4,"$240,000",,NYY
4,Ken Clay,23,us US,R,R,"6' 3""",185,"Apr 6, 1954",1st,21,3,0,21,21,0,0,0,0,0,0,0,0,0,0,0,0,-0.4,,,NYY


In [261]:
appearances.drop(columns=["WAR", "Salary", ""])

,Name,Age,B,T,Ht,Wt,DoB,Yrs,G,GS,Batting,Defense,P,C,1B,2B,3B,SS,LF,CF,RF,OF,DH,PH,PR,Tm
0,Dell Alston,24,L,R,"6' 0""",180,"Sep 22, 1952",1st,22,10,22,2,0,0,0,0,0,0,0,0,2,2,10,8,4,NYY
1,Dave Bergman,24,L,L,"6' 1""",185,"Jun 6, 1953",2,5,0,5,5,0,0,2,0,0,0,1,1,1,3,0,1,1,NYY
2,Paul Blair,33,R,R,"6' 0""",168,"Feb 1, 1944",14,83,37,83,80,0,0,0,0,0,0,6,42,34,80,3,4,4,NYY
3,Chris Chambliss,28,L,R,"6' 1""",195,"Dec 26, 1948",7,157,150,157,157,0,0,157,0,0,0,0,0,0,0,0,4,0,NYY
4,Ken Clay,23,R,R,"6' 3""",185,"Apr 6, 1954",1st,21,3,0,21,21,0,0,0,0,0,0,0,0,0,0,0,0,NYY
5,Bucky Dent,25,R,R,"5' 11""",170,"Nov 25, 1951",5,158,153,158,158,0,0,0,0,0,158,0,0,0,0,0,0,0,NYY
6,Dock Ellis,32,B,R,"6' 3""",205,"Mar 11, 1945",10,3,3,0,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NYY
7,Ed Figueroa,28,R,R,"6' 1""",190,"Oct 14, 1948",4,32,32,0,32,32,0,0,0,0,0,0,0,0,0,0,0,0,NYY
8,Ron Guidry,26,L,L,"5' 11""",161,"Aug 28, 1950",3,36,25,5,31,31,0,0,0,0,0,0,0,0,0,0,0,5,NYY
9,Don Gullett,26,R,L,"6' 0""",190,"Jan 6, 1951",8,22,22,0,22,22,0,0,0,0,0,0,0,0,0,0,0,0,NYY


In [262]:
# Now create function to pull appearances by team for all teams into one dataframe
base_app_url = 'https://www.baseball-reference.com/teams/'

def get_apps(year, teams_list):
    app = pd.DataFrame()
    team_count = 0
    for team in teams_list:
        one_team = pullTable(base_app_url + team + '/' + year + '.shtml', 'appearances')
        one_team = one_team.drop(columns=["", "WAR", "Salary"], errors='ignore')
        one_team["Tm"] = team        
        app = app.append(one_team, ignore_index=True)
        time.sleep(5)
        team_count += 1
        
    return app
        

In [263]:
appearances = get_apps(year, my_teams)

In [264]:
appearances.head()

,Name,Age,B,T,Ht,Wt,DoB,Yrs,G,GS,Batting,Defense,P,C,1B,2B,3B,SS,LF,CF,RF,OF,DH,PH,PR,Tm
0,Doug Bird,27,R,R,"6' 4""",180,"Mar 5, 1950",5,53,5,1,53,53,0,0,0,0,0,0,0,0,0,0,0,0,KCR
1,George Brett HOF,24,L,R,"6' 0""",185,"May 15, 1953",5,139,135,139,135,0,0,0,0,135,1,0,0,0,0,3,3,1,KCR
2,Jim Colborn,31,R,R,"6' 0""",190,"May 22, 1946",9,37,36,1,36,36,0,0,0,0,0,0,0,0,0,0,0,0,KCR
3,Al Cowens,25,R,R,"6' 1""",197,"Oct 25, 1951",4,162,151,162,158,0,0,0,0,0,0,0,27,142,158,2,6,0,KCR
4,Larry Gura,29,B,L,"6' 0""",170,"Nov 26, 1947",8,52,6,0,52,52,0,0,0,0,0,0,0,0,0,0,0,0,KCR


In [265]:
appearances.tail()

,Name,Age,B,T,Ht,Wt,DoB,Yrs,G,GS,Batting,Defense,P,C,1B,2B,3B,SS,LF,CF,RF,OF,DH,PH,PR,Tm
979,Pete Vuckovich,24,R,R,"6' 4""",215,"Oct 27, 1952",3,53,8,0,53,53,0,0,0,0,0,0,0,0,0,0,0,0,TOR
980,Ernie Whitt,25,L,R,"6' 2""",200,"Jun 13, 1952",2,23,10,23,14,0,14,0,0,0,0,0,0,0,0,0,9,2,TOR
981,Mike Willis,26,L,L,"6' 2""",205,"Dec 26, 1950",1st,43,3,0,43,43,0,0,0,0,0,0,0,0,0,0,0,0,TOR
982,Al Woods,23,L,L,"6' 3""",190,"Aug 8, 1953",1st,122,113,122,115,0,0,0,0,0,0,107,0,14,115,5,6,2,TOR
983,Gary Woods,23,R,R,"6' 2""",185,"Jul 20, 1953",2,60,57,60,60,0,0,0,0,0,0,0,60,0,60,0,1,0,TOR


In [266]:
findTables(url_bat)

['teams_standard_batting', 'players_standard_batting']

In [267]:
bat = pullTable(url_bat, 'players_standard_batting')

In [268]:
findTables(url_pit)

['teams_standard_pitching', 'players_standard_pitching']

In [269]:
pit = pullTable(url_pit, 'players_standard_pitching')

In [270]:
findTables(url_fld)

['teams_standard_fielding', 'players_players_standard_fielding_fielding']

In [271]:
fld = pullTable(url_fld, 'players_players_standard_fielding_fielding')

In [272]:
findTables(url_cat)

['teams_standard_fielding',
 'teams_advanced_fielding_c',
 'teams_advanced_fielding_c_baserunning',
 'players_players_standard_fielding_fielding',
 'players_players_advanced_fielding_c_fielding',
 'players_players_advanced_fielding_c_baserunning_fielding']

In [273]:
cat = pullTable(url_cat, 'players_players_standard_fielding_fielding')

In [274]:
bat.tail()

,Rk,Name,Age,Tm,Lg,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Pos Summary
855,856,Pat Zachry,25,NYM,NL,19,43,42,0,6,0,0,0,1,0,0,0,13,.143,.143,.143,.286,-21,6,4,0,1,0,0,1
856,857,Joe Zdeb,24,KCR,AL,105,217,195,26,58,5,2,2,23,6,5,16,23,.297,.346,.374,.720,96,73,5,0,3,3,3,7H/9D58
857,858,George Zeber#,26,NYY,AL,25,75,65,8,21,3,0,3,10,0,0,9,11,.323,.405,.508,.913,149,33,1,0,1,0,1,4/56DH
858,859,Richie Zisk,28,CHW,AL,141,599,531,78,154,17,6,30,101,0,4,55,98,.290,.355,.514,.869,134,273,15,3,1,9,7,*9D/7H
859,,LgAvg per 600 PA,,,,190,600,535,69,141,24,4,13,65,11,6,51,82,.263,.328,.399,.726,,213,12,3,7,5,5,


In [275]:
bat = bat[bat["Name"] != "LgAvg per 600 PA"]

In [276]:
def how_bats(names):
    bats = ""
    for name in names:
        if name.rfind("#") > -1:
            bats = "S"
        elif name.rfind("*") > -1:
            bats = "L"
        else:
            bats = "R"
    return bats

In [277]:
bat["Bats"] = bat["Name"].apply(how_bats)
bat["Bats"].value_counts()

R    539
L    245
S     75
Name: Bats, dtype: int64

In [278]:
bat["Name"] = [re.sub("[*#]", "", name) for name in bat["Name"]]
bat.tail()

,Rk,Name,Age,Tm,Lg,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Pos Summary,Bats
854,855,Pat Zachry,25,CIN,NL,12,27,22,0,3,0,0,0,0,0,0,1,11,.136,.174,.136,.310,-16,3,0,0,4,0,0,1,R
855,856,Pat Zachry,25,NYM,NL,19,43,42,0,6,0,0,0,1,0,0,0,13,.143,.143,.143,.286,-21,6,4,0,1,0,0,1,R
856,857,Joe Zdeb,24,KCR,AL,105,217,195,26,58,5,2,2,23,6,5,16,23,.297,.346,.374,.720,96,73,5,0,3,3,3,7H/9D58,R
857,858,George Zeber,26,NYY,AL,25,75,65,8,21,3,0,3,10,0,0,9,11,.323,.405,.508,.913,149,33,1,0,1,0,1,4/56DH,S
858,859,Richie Zisk,28,CHW,AL,141,599,531,78,154,17,6,30,101,0,4,55,98,.290,.355,.514,.869,134,273,15,3,1,9,7,*9D/7H,R


In [279]:
bat.drop(columns=["Rk"], inplace=True)

In [280]:
bat.tail()

,Name,Age,Tm,Lg,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Pos Summary,Bats
854,Pat Zachry,25,CIN,NL,12,27,22,0,3,0,0,0,0,0,0,1,11,.136,.174,.136,.310,-16,3,0,0,4,0,0,1,R
855,Pat Zachry,25,NYM,NL,19,43,42,0,6,0,0,0,1,0,0,0,13,.143,.143,.143,.286,-21,6,4,0,1,0,0,1,R
856,Joe Zdeb,24,KCR,AL,105,217,195,26,58,5,2,2,23,6,5,16,23,.297,.346,.374,.720,96,73,5,0,3,3,3,7H/9D58,R
857,George Zeber,26,NYY,AL,25,75,65,8,21,3,0,3,10,0,0,9,11,.323,.405,.508,.913,149,33,1,0,1,0,1,4/56DH,S
858,Richie Zisk,28,CHW,AL,141,599,531,78,154,17,6,30,101,0,4,55,98,.290,.355,.514,.869,134,273,15,3,1,9,7,*9D/7H,R


In [281]:
pit.tail()

,Rk,Name,Age,Tm,Lg,W,L,W-L%,ERA,G,GS,GF,CG,SHO,SV,IP,H,R,ER,HR,BB,IBB,SO,HBP,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W
454,455,Pat Zachry,25,TOT,NL,10,13,.435,4.25,31,31,0,5,1,0,194.2,207,104,92,21,77,5,99,4,1,0,845,90,4.37,1.459,9.6,1.0,3.6,4.6,1.29
455,456,Pat Zachry,25,CIN,NL,3,7,.300,5.04,12,12,0,3,0,0,75.0,78,45,42,7,29,1,36,1,0,0,321,78,4.19,1.427,9.4,0.8,3.5,4.3,1.24
456,457,Pat Zachry,25,NYM,NL,7,6,.538,3.76,19,19,0,2,1,0,119.2,129,59,50,14,48,4,63,3,1,0,524,99,4.49,1.479,9.7,1.1,3.6,4.7,1.31
457,458,Geoff Zahn*,31,MIN,AL,12,14,.462,4.68,34,32,0,7,1,0,198.0,234,116,103,20,66,4,88,5,0,5,870,85,4.24,1.515,10.6,0.9,3.0,4.0,1.33
458,,LgAvg per 180 IP,,,,10,10,.499,4.04,51,20,16,4,1,4,180,182,91,81,18,66,6,103,4,1,6,772,99,4.03,1.378,9.1,0.9,3.3,5.1,1.56


In [282]:
pit = pit[pit["Name"] != "LgAvg per 180 IP"]

In [283]:
def how_throws(names):
    throws = ""
    for name in names:
        if name.rfind("*") > -1:
            throws = "L"
        else:
            throws = "R"
    return throws

In [284]:
pit["Throws"] = pit["Name"].apply(how_throws)
pit["Throws"].value_counts()

R    315
L    143
Name: Throws, dtype: int64

In [285]:
pit["Name"] = [re.sub("[*#]", "", name) for name in pit["Name"]]
pit.tail()

,Rk,Name,Age,Tm,Lg,W,L,W-L%,ERA,G,GS,GF,CG,SHO,SV,IP,H,R,ER,HR,BB,IBB,SO,HBP,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws
453,454,Wilbur Wood,35,CHW,AL,7,8,.467,4.99,24,18,2,5,1,0,122.2,139,75,68,10,50,0,42,10,0,0,548,83,4.58,1.541,10.2,0.7,3.7,3.1,0.84,L
454,455,Pat Zachry,25,TOT,NL,10,13,.435,4.25,31,31,0,5,1,0,194.2,207,104,92,21,77,5,99,4,1,0,845,90,4.37,1.459,9.6,1.0,3.6,4.6,1.29,R
455,456,Pat Zachry,25,CIN,NL,3,7,.300,5.04,12,12,0,3,0,0,75.0,78,45,42,7,29,1,36,1,0,0,321,78,4.19,1.427,9.4,0.8,3.5,4.3,1.24,R
456,457,Pat Zachry,25,NYM,NL,7,6,.538,3.76,19,19,0,2,1,0,119.2,129,59,50,14,48,4,63,3,1,0,524,99,4.49,1.479,9.7,1.1,3.6,4.7,1.31,R
457,458,Geoff Zahn,31,MIN,AL,12,14,.462,4.68,34,32,0,7,1,0,198.0,234,116,103,20,66,4,88,5,0,5,870,85,4.24,1.515,10.6,0.9,3.0,4.0,1.33,L


In [286]:
pit.drop(columns=["Rk"], inplace=True)

In [287]:
players = pd.merge(bat, pit, how="outer", on=["Name", "Tm", "Age"], suffixes=('_bat', '_pit'))

In [288]:
players.shape

(1067, 62)

In [289]:
fld.tail()

,Rk,Name,Age,Tm,Lg,G,GS,CG,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos Summary
886,887,Geoff Zahn,31,MIN,AL,34,32,7,198.0,58,16,40,2,5,.966,,,2.55,1.65,P
887,888,Joe Zdeb,24,KCR,AL,94,41,31,451.1,100,93,4,3,0,.970,1,2,1.93,1.03,OF-3B
888,889,George Zeber,26,NYY,AL,25,16,12,164.1,102,42,56,4,11,.961,-1,-7,5.37,3.92,2B-3B-SS
889,890,Richie Zisk,28,CHW,AL,109,108,97,931.1,223,210,9,4,3,.982,-9,-11,2.12,2.01,OF
890,,LgAvg,,,,198,154,120,1379,669,465,189,15,42,.978,0,0,4.27,3.29,


In [290]:
fld = fld[fld["Name"] != "LgAvg"]

In [291]:
fld.drop(columns=["Rk"], inplace=True)

In [292]:
players = pd.merge(players, fld, how="left", on=["Name", "Age"], suffixes=('', '_fld'))

In [293]:
cat.tail()

,Rk,Name,Age,Tm,Lg,G,GS,CG,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rctch,RF/9,RF/G,PB,WP,SB,CS,CS%
96,97,Earl Williams,28,OAK,AL,36,33,16,269.0,188,165,21,2,2,.989,2,9,2,6.22,5.17,2,10,26,20,43%
97,98,John Wockenfuss,28,DET,AL,37,33,31,296.2,198,175,20,3,2,.985,0,0,0,5.92,5.27,5,4,20,12,38%
98,99,Butch Wynegar,21,MIN,AL,142,138,135,1247.0,765,676,84,5,8,.993,4,4,4,5.49,5.35,10,41,81,60,43%
99,100,Steve Yeager,28,LAD,NL,123,119,90,1015.0,797,690,89,18,12,.977,6,7,6,6.91,6.33,6,19,56,36,39%
100,,LgAvg,,,,20,17,15,156,110,97,11,2,1,.985,0,0,0,6.21,5.29,1,5,12,7,37%


In [294]:
cat.drop_duplicates(subset=["Name"], keep='first', inplace=True)

In [295]:
cat = cat[cat["Name"] != "LgAvg"]

In [296]:
cat.drop(columns=["Rk"], inplace=True)

In [297]:
players = pd.merge(players, cat, how='left', on=["Name", "Age"], suffixes=('', '_cat'))

In [298]:
appearances.tail()

,Name,Age,B,T,Ht,Wt,DoB,Yrs,G,GS,Batting,Defense,P,C,1B,2B,3B,SS,LF,CF,RF,OF,DH,PH,PR,Tm
979,Pete Vuckovich,24,R,R,"6' 4""",215,"Oct 27, 1952",3,53,8,0,53,53,0,0,0,0,0,0,0,0,0,0,0,0,TOR
980,Ernie Whitt,25,L,R,"6' 2""",200,"Jun 13, 1952",2,23,10,23,14,0,14,0,0,0,0,0,0,0,0,0,9,2,TOR
981,Mike Willis,26,L,L,"6' 2""",205,"Dec 26, 1950",1st,43,3,0,43,43,0,0,0,0,0,0,0,0,0,0,0,0,TOR
982,Al Woods,23,L,L,"6' 3""",190,"Aug 8, 1953",1st,122,113,122,115,0,0,0,0,0,0,107,0,14,115,5,6,2,TOR
983,Gary Woods,23,R,R,"6' 2""",185,"Jul 20, 1953",2,60,57,60,60,0,0,0,0,0,0,0,60,0,60,0,1,0,TOR


In [299]:
appearances.columns

Index(['Name', 'Age', 'B', 'T', 'Ht', 'Wt', 'DoB', 'Yrs', 'G', 'GS', 'Batting',
       'Defense', 'P', 'C', '1B', '2B', '3B', 'SS', 'LF', 'CF', 'RF', 'OF',
       'DH', 'PH', 'PR', 'Tm'],
      dtype='object')

In [300]:
appearances = appearances.drop(columns=["", "B", "T", "Ht", "Wt", "DoB", "Yrs", "WAR", "Salary", ""], axis=1,
                               errors='ignore')
appearances.head()

,Name,Age,G,GS,Batting,Defense,P,C,1B,2B,3B,SS,LF,CF,RF,OF,DH,PH,PR,Tm
0,Doug Bird,27,53,5,1,53,53,0,0,0,0,0,0,0,0,0,0,0,0,KCR
1,George Brett HOF,24,139,135,139,135,0,0,0,0,135,1,0,0,0,0,3,3,1,KCR
2,Jim Colborn,31,37,36,1,36,36,0,0,0,0,0,0,0,0,0,0,0,0,KCR
3,Al Cowens,25,162,151,162,158,0,0,0,0,0,0,0,27,142,158,2,6,0,KCR
4,Larry Gura,29,52,6,0,52,52,0,0,0,0,0,0,0,0,0,0,0,0,KCR


In [301]:
players.head()

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%
0,Bob Adams,25,DET,AL,15,24,24,2,6,1,0,2,2,0,0,0,5,.250,.250,.542,.792,105,13,0,0,0,0,0,H/32,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DET,AL,3,3,3,27.0,27,26,1,0,1,1.000,-1,-22,9.00,9.00,1B-C,DET,AL,1,1,1,9.0,6,6,0,0,0,1.000,0,0,0,6.00,6.00,0,0,1,0,0%
1,Glenn Adams,29,MIN,AL,95,290,269,32,91,17,0,6,49,0,2,18,30,.338,.376,.468,.844,130,126,5,0,0,3,3,D97H,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIN,AL,44,38,11,271.0,65,60,3,2,1,.969,-3,-13,2.09,1.43,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mike Adams,28,CHC,NL,2,2,2,0,0,0,0,0,0,0,0,0,1,.000,.000,.000,.000,-100,0,0,0,0,0,0,/87H,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHC,NL,2,1,0,2.0,0,0,0,0,0,,0,-120,0.00,0.00,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Willie Aikens,22,CAL,AL,42,101,91,5,18,4,0,0,6,1,2,10,23,.198,.277,.242,.519,46,22,1,0,0,0,2,H3D,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAL,AL,13,12,6,99.0,105,94,8,3,10,.971,0,-2,9.27,7.85,1B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Santo Alcala,24,TOT,NL,38,30,28,1,2,0,0,1,2,0,0,0,15,.071,.071,.179,.250,-34,5,0,0,2,0,0,1,R,NL,3,7,.300,4.83,38,12,8,0,0,2,117.1,126,66,63,13,54,6,73,3,3,6,520,79,4.39,1.534,9.7,1.0,4.1,5.6,1.35,R,TOT,NL,38,12,0,117.1,19,8,11,0,0,1.000,,,1.46,0.50,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [302]:
players.shape

(1067, 101)

In [303]:
pd.set_option('display.max_rows', 1000)
players

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%
0,Bob Adams,25,DET,AL,15,24,24,2,6,1,0,2,2,0,0,0,5,.250,.250,.542,.792,105,13,0,0,0,0,0,H/32,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DET,AL,3,3,3,27.0,27,26,1,0,1,1.000,-1,-22,9.00,9.00,1B-C,DET,AL,1,1,1,9.0,6,6,0,0,0,1.000,0,0,0,6.00,6.00,0,0,1,0,0%
1,Glenn Adams,29,MIN,AL,95,290,269,32,91,17,0,6,49,0,2,18,30,.338,.376,.468,.844,130,126,5,0,0,3,3,D97H,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIN,AL,44,38,11,271.0,65,60,3,2,1,.969,-3,-13,2.09,1.43,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mike Adams,28,CHC,NL,2,2,2,0,0,0,0,0,0,0,0,0,1,.000,.000,.000,.000,-100,0,0,0,0,0,0,/87H,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHC,NL,2,1,0,2.0,0,0,0,0,0,,0,-120,0.00,0.00,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Willie Aikens,22,CAL,AL,42,101,91,5,18,4,0,0,6,1,2,10,23,.198,.277,.242,.519,46,22,1,0,0,0,2,H3D,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAL,AL,13,12,6,99.0,105,94,8,3,10,.971,0,-2,9.27,7.85,1B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Santo Alcala,24,TOT,NL,38,30,28,1,2,0,0,1,2,0,0,0,15,.071,.071,.179,.250,-34,5,0,0,2,0,0,1,R,NL,3,7,.300,4.83,38,12,8,0,0,2,117.1,126,66,63,13,54,6,73,3,3,6,520,79,4.39,1.534,9.7,1.0,4.1,5.6,1.35,R,TOT,NL,38,12,0,117.1,19,8,11,0,0,1.000,,,1.46,0.50,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062,Mike Willis,26,TOR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,2,6,.250,3.94,43,3,28,0,0,5,107.1,105,48,47,15,38,6,59,0,0,2,451,107,4.52,1.332,8.8,1.3,3.2,4.9,1.55,L,TOR,AL,43,3,0,107.1,26,6,19,1,1,.962,,,2.10,0.58,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1063,Jim Willoughby,28,BOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,6,2,.750,4.94,31,0,16,0,0,2,54.2,54,32,30,5,18,3,33,2,0,2,234,92,3.82,1.317,8.9,0.8,3.0,5.4,1.83,R,BOS,AL,31,0,0,54.2,23,12,10,1,1,.957,,,3.62,0.71,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1064,Rick Wise,31,BOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,11,5,.688,4.77,26,20,2,4,2,0,128.1,151,68,68,19,28,1,85,4,1,1,555,95,4.09,1.395,10.6,1.3,2.0,6.0,3.04,R,BOS,AL,26,20,4,128.1,31,8,23,0,1,1.000,,,2.17,1.19,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1065,Wilbur Wood,35,CHW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [304]:
players["Name"] = players["Name"].str.strip()

In [305]:
players["Name"] = [' '.join(x.split()) for x in players["Name"]]

In [306]:
players.head()

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%
0,Bob Adams,25,DET,AL,15,24,24,2,6,1,0,2,2,0,0,0,5,.250,.250,.542,.792,105,13,0,0,0,0,0,H/32,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DET,AL,3,3,3,27.0,27,26,1,0,1,1.000,-1,-22,9.00,9.00,1B-C,DET,AL,1,1,1,9.0,6,6,0,0,0,1.000,0,0,0,6.00,6.00,0,0,1,0,0%
1,Glenn Adams,29,MIN,AL,95,290,269,32,91,17,0,6,49,0,2,18,30,.338,.376,.468,.844,130,126,5,0,0,3,3,D97H,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIN,AL,44,38,11,271.0,65,60,3,2,1,.969,-3,-13,2.09,1.43,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mike Adams,28,CHC,NL,2,2,2,0,0,0,0,0,0,0,0,0,1,.000,.000,.000,.000,-100,0,0,0,0,0,0,/87H,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHC,NL,2,1,0,2.0,0,0,0,0,0,,0,-120,0.00,0.00,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Willie Aikens,22,CAL,AL,42,101,91,5,18,4,0,0,6,1,2,10,23,.198,.277,.242,.519,46,22,1,0,0,0,2,H3D,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAL,AL,13,12,6,99.0,105,94,8,3,10,.971,0,-2,9.27,7.85,1B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Santo Alcala,24,TOT,NL,38,30,28,1,2,0,0,1,2,0,0,0,15,.071,.071,.179,.250,-34,5,0,0,2,0,0,1,R,NL,3,7,.300,4.83,38,12,8,0,0,2,117.1,126,66,63,13,54,6,73,3,3,6,520,79,4.39,1.534,9.7,1.0,4.1,5.6,1.35,R,TOT,NL,38,12,0,117.1,19,8,11,0,0,1.000,,,1.46,0.50,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [307]:
appearances["Name"] = [' '.join(x.split()) for x in appearances["Name"]]
appearances.head()

,Name,Age,G,GS,Batting,Defense,P,C,1B,2B,3B,SS,LF,CF,RF,OF,DH,PH,PR,Tm
0,Doug Bird,27,53,5,1,53,53,0,0,0,0,0,0,0,0,0,0,0,0,KCR
1,George Brett HOF,24,139,135,139,135,0,0,0,0,135,1,0,0,0,0,3,3,1,KCR
2,Jim Colborn,31,37,36,1,36,36,0,0,0,0,0,0,0,0,0,0,0,0,KCR
3,Al Cowens,25,162,151,162,158,0,0,0,0,0,0,0,27,142,158,2,6,0,KCR
4,Larry Gura,29,52,6,0,52,52,0,0,0,0,0,0,0,0,0,0,0,0,KCR


In [308]:
appearances["Name"] = appearances["Name"].str.replace(" HOF", "")
appearances

,Name,Age,G,GS,Batting,Defense,P,C,1B,2B,3B,SS,LF,CF,RF,OF,DH,PH,PR,Tm
0,Doug Bird,27,53,5,1,53,53,0,0,0,0,0,0,0,0,0,0,0,0,KCR
1,George Brett,24,139,135,139,135,0,0,0,0,135,1,0,0,0,0,3,3,1,KCR
2,Jim Colborn,31,37,36,1,36,36,0,0,0,0,0,0,0,0,0,0,0,0,KCR
3,Al Cowens,25,162,151,162,158,0,0,0,0,0,0,0,27,142,158,2,6,0,KCR
4,Larry Gura,29,52,6,0,52,52,0,0,0,0,0,0,0,0,0,0,0,0,KCR
5,Tom Hall,29,6,0,0,6,6,0,0,0,0,0,0,0,0,0,0,0,0,KCR
6,Andy Hassler,25,30,28,1,29,29,0,0,0,0,0,0,0,0,0,0,0,0,KCR
7,Bob Heise,30,54,16,54,52,0,0,0,23,11,21,0,0,0,0,0,1,8,KCR
8,Clint Hurdle,19,9,6,9,9,0,0,0,0,0,0,0,0,9,9,0,1,0,KCR
9,Pete LaCock,25,88,54,88,41,0,0,29,0,0,0,9,0,3,12,26,24,0,KCR


In [309]:
players = pd.merge(players, appearances, how='left', on=["Name", "Tm", "Age"], suffixes=('', '_app'))

In [310]:
players.head()

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR
0,Bob Adams,25,DET,AL,15,24,24,2,6,1,0,2,2,0,0,0,5,.250,.250,.542,.792,105,13,0,0,0,0,0,H/32,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DET,AL,3,3,3,27.0,27,26,1,0,1,1.000,-1,-22,9.00,9.00,1B-C,DET,AL,1,1,1,9.0,6,6,0,0,0,1.000,0,0,0,6.00,6.00,0,0,1,0,0%,15,3,15,3,0,1,2,0,0,0,0,0,0,0,0,12,0
1,Glenn Adams,29,MIN,AL,95,290,269,32,91,17,0,6,49,0,2,18,30,.338,.376,.468,.844,130,126,5,0,0,3,3,D97H,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIN,AL,44,38,11,271.0,65,60,3,2,1,.969,-3,-13,2.09,1.43,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95,80,95,44,0,0,0,0,0,0,16,0,28,44,46,14,0
2,Mike Adams,28,CHC,NL,2,2,2,0,0,0,0,0,0,0,0,0,1,.000,.000,.000,.000,-100,0,0,0,0,0,0,/87H,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHC,NL,2,1,0,2.0,0,0,0,0,0,,0,-120,0.00,0.00,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,2,2,0,0,0,0,0,0,1,1,0,2,NaN,1,0
3,Willie Aikens,22,CAL,AL,42,101,91,5,18,4,0,0,6,1,2,10,23,.198,.277,.242,.519,46,22,1,0,0,0,2,H3D,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAL,AL,13,12,6,99.0,105,94,8,3,10,.971,0,-2,9.27,7.85,1B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42,23,42,13,0,0,13,0,0,0,0,0,0,0,13,18,0
4,Santo Alcala,24,TOT,NL,38,30,28,1,2,0,0,1,2,0,0,0,15,.071,.071,.179,.250,-34,5,0,0,2,0,0,1,R,NL,3,7,.300,4.83,38,12,8,0,0,2,117.1,126,66,63,13,54,6,73,3,3,6,520,79,4.39,1.534,9.7,1.0,4.1,5.6,1.35,R,TOT,NL,38,12,0,117.1,19,8,11,0,0,1.000,,,1.46,0.50,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [311]:
players.to_csv("../data/player stats" + " - " + year + ".csv", index=False)

# Rate Players

In [312]:
players = pd.read_csv("../data/player stats - " + year + ".csv")
players.head()

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR
0,Bob Adams,25,DET,AL,15.0,24.0,24.0,2.0,6.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,5.0,0.250,0.250,0.542,0.792,105.0,13.0,0.0,0.0,0.0,0.0,0.0,H/32,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DET,AL,3.0,3.0,3.0,27.0,27.0,26.0,1.0,0.0,1.0,1.000,-1.0,-22.0,9.00,9.00,1B-C,DET,AL,1.0,1.0,1.0,9.0,6.0,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0,6.0,0.0,0.0,1.0,0.0,0%,15.0,3.0,15.0,3.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0
1,Glenn Adams,29,MIN,AL,95.0,290.0,269.0,32.0,91.0,17.0,0.0,6.0,49.0,0.0,2.0,18.0,30.0,0.338,0.376,0.468,0.844,130.0,126.0,5.0,0.0,0.0,3.0,3.0,D97H,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIN,AL,44.0,38.0,11.0,271.0,65.0,60.0,3.0,2.0,1.0,0.969,-3.0,-13.0,2.09,1.43,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.0,80.0,95.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,28.0,44.0,46.0,14.0,0.0
2,Mike Adams,28,CHC,NL,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000,0.000,0.000,0.000,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,/87H,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHC,NL,2.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,-120.0,0.00,0.00,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,NaN,1.0,0.0
3,Willie Aikens,22,CAL,AL,42.0,101.0,91.0,5.0,18.0,4.0,0.0,0.0,6.0,1.0,2.0,10.0,23.0,0.198,0.277,0.242,0.519,46.0,22.0,1.0,0.0,0.0,0.0,2.0,H3D,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAL,AL,13.0,12.0,6.0,99.0,105.0,94.0,8.0,3.0,10.0,0.971,0.0,-2.0,9.27,7.85,1B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,23.0,42.0,13.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,18.0,0.0
4,Santo Alcala,24,TOT,NL,38.0,30.0,28.0,1.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,15.0,0.071,0.071,0.179,0.250,-34.0,5.0,0.0,0.0,2.0,0.0,0.0,1,R,NL,3.0,7.0,0.3,4.83,38.0,12.0,8.0,0.0,0.0,2.0,117.1,126.0,66.0,63.0,13.0,54.0,6.0,73.0,3.0,3.0,6.0,520.0,79.0,4.39,1.534,9.7,1.0,4.1,5.6,1.35,R,TOT,NL,38.0,12.0,0.0,117.1,19.0,8.0,11.0,0.0,0.0,1.000,NaN,NaN,1.46,0.50,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [313]:
players.tail()

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR
1062,Mike Willis,26,TOR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,2.0,6.0,0.250,3.94,43.0,3.0,28.0,0.0,0.0,5.0,107.1,105.0,48.0,47.0,15.0,38.0,6.0,59.0,0.0,0.0,2.0,451.0,107.0,4.52,1.332,8.8,1.3,3.2,4.9,1.55,L,TOR,AL,43.0,3.0,0.0,107.1,26.0,6.0,19.0,1.0,1.0,0.962,NaN,NaN,2.10,0.58,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.0,3.0,0.0,43.0,43.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1063,Jim Willoughby,28,BOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,6.0,2.0,0.750,4.94,31.0,0.0,16.0,0.0,0.0,2.0,54.2,54.0,32.0,30.0,5.0,18.0,3.0,33.0,2.0,0.0,2.0,234.0,92.0,3.82,1.317,8.9,0.8,3.0,5.4,1.83,R,BOS,AL,31.0,0.0,0.0,54.2,23.0,12.0,10.0,1.0,1.0,0.957,NaN,NaN,3.62,0.71,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0,0.0,0.0,31.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1064,Rick Wise,31,BOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,11.0,5.0,0.688,4.77,26.0,20.0,2.0,4.0,2.0,0.0,128.1,151.0,68.0,68.0,19.0,28.0,1.0,85.0,4.0,1.0,1.0,555.0,95.0,4.09,1.395,10.6,1.3,2.0,6.0,3.04,R,BOS,AL,26.0,20.0,4.0,128.1,31.0,8.0,23.0,0.0,1.0,1.000,NaN,NaN,2.17,1.19,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0,20.0,0.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1065,Wilbur Wood,35,CHW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,7.0,8.0,0.467,4.99,24.0,18.0,2.0,5.0,1.0,0.0,122.2,139.0,75.0,68.0,10.0,50.0,0.0,42.0,10.0,0.0,0.0,548.0,83.0,4.58,1.541,10.2,0.7,3.7,3.1,0.84,L,CHW,AL,24.0,18.0,5.0,122.2,36.0,3.0,33.0,0.0,1.0,1.000,NaN,NaN,2.64,1.50,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,18.0,0.0,24.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1066,Geoff Zahn,31,MIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,12.0,14.0,0.462,4.68,34.0,32.0,0.0,7.0,1.0,0.0,198.0,234.0,116.0,103.0,20.0,66.0,4.0,88.0,5.0,0.0,5.0,870.0,85.0,4.24,1.515,10.6,0.9,3.0,4.0,1.33,L,MIN,AL,34.0,32.0,7.0,198.0,58.0,16.0,40.0,2.0,5.0,0.966,NaN,NaN,2.55,1.65,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0,32.0,0.0,34.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [314]:
pd.set_option('max_seq_items', 200)
players.columns

Index(['Name', 'Age', 'Tm', 'Lg_bat', 'G_bat', 'PA', 'AB', 'R_bat', 'H_bat',
       '2B', '3B', 'HR_bat', 'RBI', 'SB', 'CS', 'BB_bat', 'SO_bat', 'BA',
       'OBP', 'SLG', 'OPS', 'OPS+', 'TB', 'GDP', 'HBP_bat', 'SH', 'SF',
       'IBB_bat', 'Pos Summary', 'Bats', 'Lg_pit', 'W', 'L', 'W-L%', 'ERA',
       'G_pit', 'GS', 'GF', 'CG', 'SHO', 'SV', 'IP', 'H_pit', 'R_pit', 'ER',
       'HR_pit', 'BB_pit', 'IBB_pit', 'SO_pit', 'HBP_pit', 'BK', 'WP', 'BF',
       'ERA+', 'FIP', 'WHIP', 'H9', 'HR9', 'BB9', 'SO9', 'SO/W', 'Throws',
       'Tm_fld', 'Lg', 'G', 'GS_fld', 'CG_fld', 'Inn', 'Ch', 'PO', 'A', 'E',
       'DP', 'Fld%', 'Rtot', 'Rtot/yr', 'RF/9', 'RF/G', 'Pos Summary_fld',
       'Tm_cat', 'Lg_cat', 'G_cat', 'GS_cat', 'CG_cat', 'Inn_cat', 'Ch_cat',
       'PO_cat', 'A_cat', 'E_cat', 'DP_cat', 'Fld%_cat', 'Rtot_cat',
       'Rtot/yr_cat', 'Rctch', 'RF/9_cat', 'RF/G_cat', 'PB', 'WP_cat',
       'SB_cat', 'CS_cat', 'CS%', 'G_app', 'GS_app', 'Batting', 'Defense', 'P',
       'C', '1B', '2B_a

In [315]:
list(players.columns.values)

['Name',
 'Age',
 'Tm',
 'Lg_bat',
 'G_bat',
 'PA',
 'AB',
 'R_bat',
 'H_bat',
 '2B',
 '3B',
 'HR_bat',
 'RBI',
 'SB',
 'CS',
 'BB_bat',
 'SO_bat',
 'BA',
 'OBP',
 'SLG',
 'OPS',
 'OPS+',
 'TB',
 'GDP',
 'HBP_bat',
 'SH',
 'SF',
 'IBB_bat',
 'Pos\xa0Summary',
 'Bats',
 'Lg_pit',
 'W',
 'L',
 'W-L%',
 'ERA',
 'G_pit',
 'GS',
 'GF',
 'CG',
 'SHO',
 'SV',
 'IP',
 'H_pit',
 'R_pit',
 'ER',
 'HR_pit',
 'BB_pit',
 'IBB_pit',
 'SO_pit',
 'HBP_pit',
 'BK',
 'WP',
 'BF',
 'ERA+',
 'FIP',
 'WHIP',
 'H9',
 'HR9',
 'BB9',
 'SO9',
 'SO/W',
 'Throws',
 'Tm_fld',
 'Lg',
 'G',
 'GS_fld',
 'CG_fld',
 'Inn',
 'Ch',
 'PO',
 'A',
 'E',
 'DP',
 'Fld%',
 'Rtot',
 'Rtot/yr',
 'RF/9',
 'RF/G',
 'Pos\xa0Summary_fld',
 'Tm_cat',
 'Lg_cat',
 'G_cat',
 'GS_cat',
 'CG_cat',
 'Inn_cat',
 'Ch_cat',
 'PO_cat',
 'A_cat',
 'E_cat',
 'DP_cat',
 'Fld%_cat',
 'Rtot_cat',
 'Rtot/yr_cat',
 'Rctch',
 'RF/9_cat',
 'RF/G_cat',
 'PB',
 'WP_cat',
 'SB_cat',
 'CS_cat',
 'CS%',
 'G_app',
 'GS_app',
 'Batting',
 'Defense',
 'P',
 '

## Create primary position variable

In [316]:
players.rename(columns={
    "Pos\xa0Summary": "Pos_Summary",
    "Pos\xa0Summary_fld": "Pos_Summary_fld"
}, inplace=True)

In [317]:
players["Primary_Pos_fld"] = players['Pos_Summary_fld'].str.split("-").str[0]
players["Primary_Pos_fld"].value_counts()

P     455
OF    226
C      93
1B     78
2B     75
SS     72
3B     58
Name: Primary_Pos_fld, dtype: int64

## Batter Ratings

In [318]:
# pd.set_option('display.max_columns', 200)
# players.columns

### Clutch Rating

In [319]:
players["rbi_per_g"] = players["RBI"] / players["G_bat"]
players["rbi_per_g"].value_counts()

0.000000    206
0.166667     12
0.500000      9
0.333333      9
0.250000      9
0.090909      8
0.200000      8
0.142857      7
0.181818      6
0.066667      5
0.100000      5
0.076923      5
0.058824      5
0.400000      4
0.428571      4
0.071429      4
0.272727      4
0.183673      4
0.083333      4
0.156250      4
0.375000      4
0.153846      4
0.322581      3
0.461538      3
0.230769      3
0.045455      3
0.411765      3
0.306667      3
0.111111      3
0.416667      3
0.148148      3
0.214286      3
0.307692      3
0.208333      3
0.370130      2
0.350000      2
0.052632      2
0.346154      2
0.030303      2
0.262069      2
0.258065      2
0.133333      2
0.300000      2
0.571429      2
0.777778      2
0.057143      2
0.202703      2
0.117647      2
0.035714      2
0.583333      2
0.138889      2
0.183333      2
0.028571      2
0.120000      2
0.051282      2
0.037037      2
0.135135      2
0.450980      2
0.303571      2
0.024390      2
0.462121      2
0.055556      2
0.296296

In [320]:
players["clutch"] = (round(players["rbi_per_g"], 3) >= .6).astype(int)
players["clutch"] = players["clutch"].map({0: "", 1: "#"}).astype(str)
players["clutch"].value_counts()

     1021
#      46
Name: clutch, dtype: int64

In [321]:
players[players["clutch"] == "#"]

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos_Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch
51,Kevin Bell,21,CHW,AL,9.0,32.0,28.0,4.0,5.0,1.0,0.0,1.0,6.0,0.0,0.0,3.0,8.0,0.179,0.250,0.321,0.571,55.0,9.0,0.0,0.0,0.0,1.0,0.0,/657,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHW,AL,10.0,9.0,7.0,77.1,35.0,12.0,21.0,2.0,7.0,0.943,-1.0,-19.0,3.84,3.30,SS-3B-OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,9.0,9.0,9.0,0.0,0.0,0.0,0.0,4.0,5.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,SS,0.666667,#
53,Johnny Bench,29,CIN,NL,142.0,563.0,494.0,67.0,136.0,34.0,2.0,31.0,109.0,2.0,4.0,58.0,95.0,0.275,0.348,0.540,0.889,133.0,267.0,10.0,1.0,0.0,7.0,8.0,*2/7H359,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CIN,NL,148.0,134.0,105.0,1143.1,816.0,735.0,69.0,12.0,13.0,0.985,2.0,2.0,6.33,5.43,C-OF-1B-3B,CIN,NL,135.0,125.0,102.0,1065.1,781.0,705.0,66.0,10.0,10.0,0.987,2.0,2.0,2.0,6.51,5.71,3.0,39.0,64.0,42.0,40%,142.0,134.0,142.0,142.0,0.0,135.0,4.0,0.0,1.0,0.0,7.0,0.0,1.0,8.0,NaN,6.0,0.0,C,0.767606,#
59,Kurt Bevacqua,30,TEX,AL,39.0,104.0,96.0,13.0,32.0,7.0,2.0,5.0,28.0,0.0,1.0,6.0,13.0,0.333,0.365,0.604,0.970,159.0,58.0,6.0,0.0,0.0,2.0,1.0,H5/9734D,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TEX,AL,35.0,17.0,7.0,182.0,74.0,42.0,31.0,1.0,4.0,0.986,2.0,16.0,3.61,2.09,OF-3B-2B-1B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,18.0,39.0,30.0,0.0,0.0,5.0,5.0,11.0,0.0,6.0,0.0,8.0,14.0,3.0,16.0,3.0,OF,0.717949,#
73,Bobby Bonds,31,CAL,AL,158.0,679.0,592.0,103.0,156.0,23.0,9.0,37.0,115.0,41.0,18.0,74.0,141.0,0.264,0.342,0.520,0.862,136.0,308.0,9.0,2.0,1.0,10.0,5.0,*9D/H,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAL,AL,140.0,139.0,131.0,1222.1,281.0,272.0,5.0,4.0,0.0,0.986,5.0,5.0,2.04,1.98,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,158.0,157.0,158.0,140.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,140.0,140.0,18.0,1.0,0.0,OF,0.727848,#
88,George Brett,24,KCR,AL,139.0,627.0,564.0,105.0,176.0,32.0,13.0,22.0,88.0,14.0,12.0,55.0,24.0,0.312,0.373,0.532,0.905,142.0,300.0,12.0,2.0,3.0,3.0,9.0,*5/HD6,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KCR,AL,136.0,134.0,124.0,1185.0,461.0,115.0,325.0,21.0,33.0,0.954,16.0,16.0,3.34,3.24,3B-SS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,139.0,135.0,139.0,135.0,0.0,0.0,0.0,0.0,135.0,1.0,0.0,0.0,0.0,0.0,3.0,3.0,1.0,3B,0.633094,#
106,Jeff Burroughs,26,ATL,NL,154.0,671.0,579.0,91.0,157.0,19.0,1.0,41.0,114.0,4.0,1.0,86.0,126.0,0.271,0.362,0.520,0.882,123.0,301.0,8.0,0.0,0.0,6.0,2.0,*9/H,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATL,NL,154.0,153.0,138.0,1346.0,265.0,249.0,9.0,7.0,3.0,0.974,-26.0,-23.0,1.73,1.68,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,154.0,153.0,154.0,154.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,154.0,154.0,NaN,1.0,0.0,OF,0.740260,#
123,Rod Carew,31,MIN,AL,155.0,69

### Letter Rating

In [322]:
players["BA"].value_counts()

0.000    98
0.250    20
0.333    13
0.167    12
0.241    12
0.200    11
0.261    11
0.286     9
0.240     9
0.254     8
0.273     8
0.256     7
0.231     7
0.280     7
0.300     7
0.275     7
0.284     7
0.287     7
0.500     7
0.243     7
0.264     7
0.262     7
0.239     7
0.274     6
0.267     6
0.291     6
0.111     6
0.297     6
0.143     6
0.246     6
0.245     6
0.230     6
0.271     6
0.283     6
0.222     6
0.270     5
0.269     5
0.216     5
0.235     5
0.290     5
0.182     5
0.257     5
0.190     5
0.188     5
0.174     5
0.225     5
0.059     5
0.259     5
0.221     5
0.308     5
0.289     5
0.282     5
0.265     5
0.091     5
0.186     4
0.252     4
0.278     4
0.247     4
0.251     4
0.219     4
0.299     4
0.176     4
0.318     4
0.292     4
0.316     4
0.217     4
0.228     4
0.175     4
0.260     4
0.263     4
0.302     4
0.208     4
0.248     4
0.197     4
0.293     4
0.133     4
0.233     4
0.238     3
0.258     3
0.118     3
0.309     3
0.268     3
0.298     3
0.22

In [323]:
players["BA"].isnull().sum()

257

In [324]:
players["BA"].replace(np.nan, 0.000, inplace=True)

In [325]:
break_points = [
    0.029,
    0.057,
    0.084,
    0.112,
    0.140,
    0.168,
    0.196,
    0.223,
    0.251,
    0.279,
    0.307,
    0.335,
    0.362,
    0.390
]

letters = [
    "G",
    "G+",
    "F",
    "E",
    "E+",
    "D",
    "D+",
    "C",
    "C+",
    "B",
    "B+",
    "A",
    "A+",
    "AA",
    "AAA"
]

def batter_letter(bat_avg, breakpoints=break_points, letter_grades=letters):
    i = bisect(breakpoints, bat_avg)
    return letter_grades[i]

In [326]:
players["bat_letter"] = [batter_letter(avg) for avg in players["BA"]]
players["bat_letter"].value_counts()

G      355
B      142
C+     136
B+     108
C       71
D+      59
A       55
D       43
E+      25
F       22
E       18
AAA     16
AA       7
A+       7
G+       3
Name: bat_letter, dtype: int64

In [327]:
players[players["bat_letter"] == "AAA"]

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos_Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter
40,Rick Baldwin,24,NYM,NL,40.0,4.0,4.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.500,0.500,0.500,1.000,177.0,2.0,0.0,0.0,0.0,0.0,0.0,1,L,NL,1.0,2.0,0.333,4.45,40.0,0.0,13.0,0.0,0.0,1.0,62.2,62.0,32.0,31.0,6.0,31.0,9.0,23.0,5.0,1.0,2.0,274.0,84.0,4.97,1.484,8.9,0.9,4.5,3.3,0.74,R,NYM,NL,40.0,0.0,0.0,62.2,18.0,6.0,12.0,0.0,3.0,1.000,NaN,NaN,2.59,0.45,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,0.0,40.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,P,0.000000,,AAA
110,Mike Caldwell,28,TOT,MLB,14.0,5.0,4.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.500,0.600,1.000,1.600,318.0,4.0,0.0,0.0,0.0,0.0,0.0,1,R,MLB,5.0,8.0,0.385,4.46,35.0,12.0,10.0,2.0,0.0,1.0,119.0,126.0,69.0,59.0,7.0,44.0,8.0,49.0,2.0,2.0,5.0,522.0,91.0,3.84,1.429,9.5,0.5,3.3,3.7,1.11,L,TOT,ZZ,35.0,12.0,2.0,119.0,38.0,7.0,31.0,0.0,3.0,1.000,NaN,NaN,2.87,1.09,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P,0.000000,,AAA
111,Mike Caldwell,28,CIN,NL,14.0,5.0,4.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.500,0.600,1.000,1.600,318.0,4.0,0.0,0.0,0.0,0.0,0.0,1,R,NL,0.0,0.0,NaN,4.01,14.0,0.0,5.0,0.0,0.0,1.0,24.2,25.0,11.0,11.0,1.0,8.0,1.0,11.0,0.0,0.0,1.0,105.0,99.0,3.35,1.338,9.1,0.4,2.9,4.0,1.38,L,TOT,ZZ,35.0,12.0,2.0,119.0,38.0,7.0,31.0,0.0,3.0,1.000,NaN,NaN,2.87,1.09,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,0.0,14.0,14.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,P,0.000000,,AAA
207,Mike Easler,26,PIT,NL,10.0,19.0,18.0,3.0,8.0,2.0,0.0,1.0,5.0,0.0,0.0,0.0,1.0,0.444,0.421,0.722,1.143,198.0,13.0,0.0,0.0,0.0,1.0,0.0,/H97,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PIT,NL,4.0,3.0,3.0,30.0,7.0,7.0,0.0,0.0,0.0,1.000,0.0,12.0,2.10,1.75,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,3.0,10.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,4.0,NaN,6.0,0.0,OF,0.500000,,AAA
211,Rawly Eastwick,26,STL,NL,41.0,7.0,5.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,2.0,0.400,0.571,0.400,0.971,168.0,2.0,0.0,0.0,0.0,0.0,0.0,1,R,NL,3.0,7.0,0.300,4.70,41.0,1.0,20.0,0.0,0.0,4.0,53.2,74.0,34.0,28.0,6.0,21.0,3.0,30.0,0.0,1.0,2.0,248.0,83.0,4.25,1.770,12.4,1.0,3.5,5.0,1.43,R,TOT,NL,64.0,1.0,0.0,97.0,11.0,5.0,6.0,0.0,0.0,1.000,NaN,NaN,1.02,0.17,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.0,1.0,41.0,41.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,P,0.024390,,AAA
249,Roger Freed,31,STL,NL,49.0,95.0,83.0,10.0,33.0,2.0,1.0,5.0,21.0,0.0,0.0,11.0,9.0,0.398,0.463,0.627,1.090,191.0,52.0,2.0,0.0,0.0,1.0,0.0,H3/9,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,STL,NL,24.0,19.0,5.0,142.2,115.0,107.0,7.0,1.0,13.0,0.991,-2.0,-17.0,7.19,4.75,1B-OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.0,19.0,49.0,24.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,NaN,26.0,0.0,1B,0.428571,,AAA
459,Gene Locklear,27,NYY,AL,1.0,5.0,5.0,1.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.600,0.600,0.600,1.200,231.0,3.0,0.0,0.0,0.0,0.0,0.0,/7,L,NaN,NaN,NaN,NaN,NaN,NaN,

In [328]:
players[players["bat_letter"] == "C+"]["BA"].min()

0.22399999999999998

### HR Number

In [329]:
players["hr_rate"] = round(players["HR_bat"] / players["H_bat"] * 36, 0)
players["hr_rate"].replace(np.nan, 0, inplace=True)
players["hr_rate"] = players["hr_rate"].astype(int)
players["hr_rate"].value_counts()

0     617
2      83
1      73
4      68
3      63
5      49
6      39
7      21
9      18
8      17
12      7
10      5
18      4
24      1
17      1
11      1
Name: hr_rate, dtype: int64

In [330]:
players["hr_num_bat"] = players["hr_rate"].map({
    0: "",
    1: "11",
    2: "12",
    3: "13",
    4: "14",
    5: "15",
    6: "16",
    7: "21",
    8: "22",
    9: "23",
    10: "24",
    11: "25",
    12: "26",
    13: "31",
    14: "32",
    15: "33",
    16: "34",
    17: "35",
    18: "36",
    19: "41",
    20: "42",
    21: "43",
    22: "44",
    23: "45",
    24: "46",
    25: "51",
    26: "52",
    27: "53",
    28: "54",
    29: "55",
    30: "56",
    31: "61",
    32: "62",
    33: "63",
    34: "64",
    35: "65",
    36: "66"
}).astype(str)
players["hr_num_bat"].value_counts()

      617
12     83
11     73
14     68
13     63
15     49
16     39
21     21
23     18
22     17
26      7
24      5
36      4
35      1
46      1
25      1
Name: hr_num_bat, dtype: int64

### Triple Number

In [331]:
players["triple_rate"] = round(players["3B"] / players["H_bat"] * 36, 0)
players["triple_rate"].replace(np.nan, 0, inplace=True)
players["triple_rate"] = players["triple_rate"].astype(int)
players["triple_rate"].value_counts()

0     741
1     177
2      91
3      28
4      16
12      4
5       4
9       2
6       2
18      1
10      1
Name: triple_rate, dtype: int64

In [332]:
players.loc[(players["triple_rate"] == 0), "triple_val"] = 0
players.loc[(players["triple_rate"] > 0), "triple_val"] = players["hr_rate"] + players["triple_rate"]
players["triple_val"].value_counts()

0.0     741
3.0      56
4.0      52
5.0      47
2.0      45
6.0      40
7.0      27
8.0      17
1.0      16
9.0      12
12.0      6
10.0      5
11.0      2
18.0      1
Name: triple_val, dtype: int64

In [333]:
players["triple_num"] = players["triple_val"].map({
    0: "",
    1: "(11)",
    2: "(12)",
    3: "(13)",
    4: "(14)",
    5: "(15)",
    6: "(16)",
    7: "(21)",
    8: "(22)",
    9: "(23)",
    10: "(24)",
    11: "(25)",
    12: "(26)",
    13: "(31)",
    14: "(32)",
    15: "(33)",
    16: "(34)",
    17: "(35)",
    18: "(36)",
    19: "(41)",
    20: "(42)",
    21: "(43)",
    22: "(44)",
    23: "(45)",
    24: "(46)",
    25: "(51)",
    26: "(52)",
    27: "(53)",
    28: "(54)",
    29: "(55)",
    30: "(56)",
    31: "(61)",
    32: "(62)",
    33: "(63)",
    34: "(64)",
    35: "(65)",
    36: "(66)"
}).astype(str)
players["triple_num"].value_counts()

        741
(13)     56
(14)     52
(15)     47
(12)     45
(16)     40
(21)     27
(22)     17
(11)     16
(23)     12
(26)      6
(24)      5
(25)      2
(36)      1
Name: triple_num, dtype: int64

### Speed Rating

In [334]:
# pd.set_option('display.max_seq_items', 200)
# players.columns

In [335]:
players["speed_score"] = round(players["SB"] / ((players["H_bat"] + players["BB_bat"] + players["HBP_bat"]) - \
                                          (players["2B"] + players["3B"] + players["HR_bat"])), 3)
players["speed_score"].replace(np.nan, 0.000, inplace=True)
players["speed_score"].value_counts()

0.000    676
0.013     11
0.083      7
0.031      7
0.012      7
0.022      7
0.025      7
0.024      7
0.032      6
0.062      6
0.010      6
0.019      6
0.167      6
0.017      6
0.061      6
0.037      6
0.042      6
0.059      6
0.125      5
0.058      5
0.038      5
0.033      5
0.143      5
0.008      5
0.056      5
0.048      5
0.009      4
0.006      4
0.036      4
0.015      4
0.074      4
0.053      4
0.051      4
0.018      4
0.014      4
0.030      4
0.007      4
0.016      4
1.000      4
0.027      4
0.072      3
0.075      3
0.020      3
0.011      3
0.077      3
0.047      3
0.095      3
0.250      3
0.160      3
0.040      3
0.310      3
0.028      3
0.071      3
0.034      3
0.054      2
0.021      2
inf        2
0.081      2
0.182      2
0.110      2
0.239      2
0.131      2
0.064      2
0.198      2
0.023      2
0.316      2
0.050      2
0.029      2
0.079      2
0.098      2
0.104      2
0.060      2
0.052      2
0.227      2
0.039      2
0.177      2
0.176      2

In [336]:
speed_breaks = [
    0.075,
    0.100,
    0.200,
    0.300
]

ratings = [
    "",
    "*",
    "**",
    "***",
    "****"
]

def speed_rate(speed, breakpoints=speed_breaks, speed_rates=ratings):
    i = bisect(breakpoints, speed)
    return speed_rates[i]

In [337]:
players["speed_rating"] = [speed_rate(rate) for rate in players["speed_score"]]
players["speed_rating"].value_counts()

        914
**       70
*        36
***      24
****     23
Name: speed_rating, dtype: int64

In [338]:
players[players["speed_rating"] == "****"]

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos_Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating
8,Matt Alexander,30,OAK,AL,90.0,47.0,42.0,24.0,10.0,1.0,0.0,0.0,2.0,26.0,14.0,4.0,6.0,0.238,0.304,0.262,0.566,59.0,11.0,0.0,0.0,1.0,0.0,0.0,H8D69/745,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OAK,AL,48.0,14.0,2.0,89.2,23.0,21.0,2.0,0.0,0.0,1.000,-2.0,-32.0,2.31,0.48,OF-2B-3B-SS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.0,14.0,90.0,37.0,0.0,0.0,0.0,4.0,1.0,0.0,7.0,17.0,10.0,32.0,13.0,0.0,67.0,OF,0.022222,,C+,0,,0,0.0,,2.000,****
103,Glenn Burke,24,LAD,NL,83.0,175.0,169.0,16.0,43.0,8.0,0.0,1.0,13.0,13.0,5.0,5.0,22.0,0.254,0.280,0.320,0.600,62.0,54.0,3.0,1.0,0.0,0.0,1.0,8H/79,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LAD,NL,74.0,35.0,27.0,432.0,102.0,98.0,1.0,3.0,0.0,0.971,-7.0,-21.0,2.06,1.34,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.0,35.0,83.0,74.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,65.0,5.0,74.0,NaN,9.0,15.0,OF,0.156627,,B,1,11,0,0.0,,0.325,****
116,Joe Cannon,23,HOU,NL,9.0,17.0,17.0,3.0,2.0,2.0,0.0,0.0,1.0,1.0,1.0,0.0,5.0,0.118,0.118,0.235,0.353,-4.0,4.0,0.0,0.0,0.0,0.0,0.0,/H7,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HOU,NL,3.0,3.0,3.0,25.0,7.0,7.0,0.0,0.0,0.0,1.000,0.0,19.0,2.52,2.33,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,3.0,9.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,NaN,4.0,2.0,OF,0.111111,,E+,0,,0,0.0,,inf,****
131,Cesar Cedeno,26,HOU,NL,141.0,598.0,530.0,92.0,148.0,36.0,8.0,14.0,71.0,61.0,14.0,47.0,50.0,0.279,0.346,0.457,0.802,123.0,242.0,10.0,11.0,2.0,8.0,7.0,*8/H,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HOU,NL,137.0,136.0,123.0,1184.2,350.0,335.0,14.0,1.0,2.0,0.997,5.0,5.0,2.65,2.55,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,141.0,136.0,141.0,137.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,137.0,0.0,137.0,NaN,3.0,2.0,OF,0.503546,,B,3,13,2,5.0,(15),0.412,****
194,Miguel Dilone,22,PIT,NL,29.0,49.0,44.0,5.0,6.0,0.0,0.0,0.0,0.0,12.0,0.0,2.0,3.0,0.136,0.174,0.136,0.310,-16.0,6.0,0.0,0.0,3.0,0.0,0.0,H/789,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PIT,NL,17.0,7.0,4.0,85.0,22.0,21.0,1.0,0.0,0.0,1.000,2.0,21.0,2.33,1.29,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,7.0,29.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,7.0,2.0,17.0,NaN,12.0,5.0,OF,0.000000,,E+,0,,0,0.0,,1.500,****
195,Mike Dimmel,22,BAL,AL,25.0,6.0,5.0,8.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.000,0.000,0.000,0.000,-100.0,0.0,0.0,0.0,1.0,0.0,0.0,9H/8,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BAL,AL,23.0,0.0,0.0,38.0,15.0,14.0,1.0,0.0,0.0,1.000,1.0,22.0,3.55,0.65,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,0.0,25.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

### Base on Balls Number

In [339]:
players["walk_rate"] = round(players["BB_bat"] / players["PA"] * 36, 0)
players["walk_rate"].replace(np.nan, 0, inplace=True)
players["walk_rate"] = players["walk_rate"].astype(int)
players["walk_rate"].value_counts()

0     430
3     174
2     159
4     117
1      87
5      50
6      23
7      15
8       6
12      2
9       2
18      1
10      1
Name: walk_rate, dtype: int64

In [340]:
players["bb_num"] = players["walk_rate"].map({
    0: "n",
    1: "11",
    2: "12",
    3: "13",
    4: "14",
    5: "15",
    6: "16",
    7: "21",
    8: "22",
    9: "23",
    10: "24",
    11: "25",
    12: "26",
    13: "31",
    14: "32",
    15: "33",
    16: "34",
    17: "35",
    18: "36",
    19: "41",
    20: "42",
    21: "43",
    22: "44",
    23: "45",
    24: "46",
    25: "51",
    26: "52",
    27: "53",
    28: "54",
    29: "55",
    30: "56",
    31: "61",
    32: "62",
    33: "63",
    34: "64",
    35: "65",
    36: "66"
}).astype(str)
players["bb_num"].value_counts()

n     430
13    174
12    159
14    117
11     87
15     50
16     23
21     15
22      6
23      2
26      2
24      1
36      1
Name: bb_num, dtype: int64

### Batter K Number

In [341]:
players["k_rate"] = round(players["SO_bat"] / players["PA"] * 36, 0)
players["k_rate"].replace(np.nan, 0, inplace=True)
players["k_rate"] = players["k_rate"].astype(int)
players["k_rate"].value_counts()

0     296
4     123
5     114
6      97
7      73
3      72
8      46
2      43
9      37
18     31
12     25
10     20
36     12
14     12
15     12
11     10
13      8
16      8
17      8
24      7
21      5
1       5
20      2
27      1
Name: k_rate, dtype: int64

In [342]:
players.loc[(players["k_rate"] == 0), "k_val"] = 0
players.loc[(players["k_rate"] > 0), "k_val"] = players["walk_rate"] + players["k_rate"]
players["k_val"].value_counts()

0.0     296
9.0     100
8.0      95
7.0      94
6.0      75
10.0     67
12.0     53
5.0      47
11.0     41
18.0     34
4.0      32
15.0     21
13.0     20
14.0     19
36.0     12
3.0      10
16.0      8
24.0      8
17.0      7
21.0      6
20.0      5
19.0      4
2.0       4
27.0      3
23.0      3
22.0      2
28.0      1
Name: k_val, dtype: int64

In [343]:
players["k_num"] = players["k_val"].map({
    0: "n",
    1: "11",
    2: "12",
    3: "13",
    4: "14",
    5: "15",
    6: "16",
    7: "21",
    8: "22",
    9: "23",
    10: "24",
    11: "25",
    12: "26",
    13: "31",
    14: "32",
    15: "33",
    16: "34",
    17: "35",
    18: "36",
    19: "41",
    20: "42",
    21: "43",
    22: "44",
    23: "45",
    24: "46",
    25: "51",
    26: "52",
    27: "53",
    28: "54",
    29: "55",
    30: "56",
    31: "61",
    32: "62",
    33: "63",
    34: "64",
    35: "65",
    36: "66"
}).astype(str)
players["k_num"].value_counts()

n     296
23    100
22     95
21     94
16     75
24     67
26     53
15     47
25     41
36     34
14     32
33     21
31     20
32     19
66     12
13     10
46      8
34      8
35      7
43      6
42      5
41      4
12      4
45      3
53      3
44      2
54      1
Name: k_num, dtype: int64

### Batter HBP Rating

In [344]:
players["hbp_rate"] = round(players["HBP_bat"] / players["PA"] * 36, 0)
players["hbp_rate"].replace(np.nan, 0, inplace=True)
players["hbp_rate"] = players["hbp_rate"].astype(int)
players["hbp_rate"].value_counts()

0    1016
1      45
2       4
5       1
4       1
Name: hbp_rate, dtype: int64

In [345]:
players.loc[(players["hbp_rate"] == 0), "hbp_val"] = 0
players.loc[(players["hbp_rate"] > 0), "hbp_val"] = players["k_val"] + players["hbp_rate"]

In [346]:
players["hbp_val"].value_counts()

0.0     1016
11.0       8
10.0       7
8.0        6
6.0        6
7.0        4
12.0       4
9.0        3
16.0       2
20.0       2
13.0       2
14.0       2
21.0       1
5.0        1
19.0       1
17.0       1
1.0        1
Name: hbp_val, dtype: int64

In [347]:
players["hbp_num"] = players["hbp_val"].map({
    0: "",
    1: "/11",
    2: "/12",
    3: "/13",
    4: "/14",
    5: "/15",
    6: "/16",
    7: "/21",
    8: "/22",
    9: "/23",
    10: "/24",
    11: "/25",
    12: "/26",
    13: "/31",
    14: "/32",
    15: "/33",
    16: "/34",
    17: "/35",
    18: "/36",
    19: "/41",
    20: "/42",
    21: "/43",
    22: "/44",
    23: "/45",
    24: "/46",
    25: "/51",
    26: "/52",
    27: "/53",
    28: "/54",
    29: "/55",
    30: "/56",
    31: "/61",
    32: "/62",
    33: "/63",
    34: "/64",
    35: "/65",
    36: "/66"
}).astype(str)
players["hbp_num"].value_counts()

       1016
/25       8
/24       7
/22       6
/16       6
/21       4
/26       4
/23       3
/34       2
/42       2
/31       2
/32       2
/43       1
/15       1
/11       1
/35       1
/41       1
Name: hbp_num, dtype: int64

### Probable Hit Number

In [348]:
players["hit_rate"] = round(players["H_bat"] / players["PA"] * 36, 0)
players["hit_rate"].replace(np.nan, 0, inplace=True)
players["hit_rate"] = players["hit_rate"].astype(int)
players["hit_rate"].value_counts()

0     355
8     170
9     147
7      96
6      73
10     66
4      37
5      34
3      22
11     19
2      18
12     14
14      4
18      4
13      3
36      2
15      1
22      1
1       1
Name: hit_rate, dtype: int64

In [349]:
players["PH_num_bat"] = players["hit_rate"].map({
    0: "66",
    1: "66",
    2: "65",
    3: "64",
    4: "63",
    5: "62",
    6: "61",
    7: "56",
    8: "55",
    9: "54",
    10: "53",
    11: "52",
    12: "51",
    13: "46",
    14: "45",
    15: "44",
    16: "43",
    17: "42",
    18: "41",
    19: "36",
    20: "35",
    21: "34",
    22: "33",
    23: "32",
    24: "31",
    25: "26",
    26: "25",
    27: "24",
    28: "23",
    29: "22",
    30: "21",
    31: "16",
    32: "15",
    33: "14",
    34: "13",
    35: "12",
    36: "11"
}).astype(str)
players["PH_num_bat"].value_counts()

66    356
55    170
54    147
56     96
61     73
53     66
63     37
62     34
64     22
52     19
65     18
51     14
41      4
45      4
46      3
11      2
44      1
33      1
Name: PH_num_bat, dtype: int64

### Batter Rating

In [350]:
players.columns

Index(['Name', 'Age', 'Tm', 'Lg_bat', 'G_bat', 'PA', 'AB', 'R_bat', 'H_bat',
       '2B', '3B', 'HR_bat', 'RBI', 'SB', 'CS', 'BB_bat', 'SO_bat', 'BA',
       'OBP', 'SLG', 'OPS', 'OPS+', 'TB', 'GDP', 'HBP_bat', 'SH', 'SF',
       'IBB_bat', 'Pos_Summary', 'Bats', 'Lg_pit', 'W', 'L', 'W-L%', 'ERA',
       'G_pit', 'GS', 'GF', 'CG', 'SHO', 'SV', 'IP', 'H_pit', 'R_pit', 'ER',
       'HR_pit', 'BB_pit', 'IBB_pit', 'SO_pit', 'HBP_pit', 'BK', 'WP', 'BF',
       'ERA+', 'FIP', 'WHIP', 'H9', 'HR9', 'BB9', 'SO9', 'SO/W', 'Throws',
       'Tm_fld', 'Lg', 'G', 'GS_fld', 'CG_fld', 'Inn', 'Ch', 'PO', 'A', 'E',
       'DP', 'Fld%', 'Rtot', 'Rtot/yr', 'RF/9', 'RF/G', 'Pos_Summary_fld',
       'Tm_cat', 'Lg_cat', 'G_cat', 'GS_cat', 'CG_cat', 'Inn_cat', 'Ch_cat',
       'PO_cat', 'A_cat', 'E_cat', 'DP_cat', 'Fld%_cat', 'Rtot_cat',
       'Rtot/yr_cat', 'Rctch', 'RF/9_cat', 'RF/G_cat', 'PB', 'WP_cat',
       'SB_cat', 'CS_cat', 'CS%', 'G_app', 'GS_app', 'Batting', 'Defense', 'P',
       'C', '1B', '2B_a

In [351]:
players["batter_rating"] = (players["clutch"] + players["bat_letter"] + \
                            players["hr_num_bat"] + players["triple_num"] + \
                            players["speed_rating"] + " [" + players["bb_num"] + \
                            "-" + players["k_num"] + players["hbp_num"] + "]")
players.head()

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos_Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating
0,Bob Adams,25,DET,AL,15.0,24.0,24.0,2.0,6.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,5.0,0.250,0.250,0.542,0.792,105.0,13.0,0.0,0.0,0.0,0.0,0.0,H/32,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DET,AL,3.0,3.0,3.0,27.0,27.0,26.0,1.0,0.0,1.0,1.000,-1.0,-22.0,9.00,9.00,1B-C,DET,AL,1.0,1.0,1.0,9.0,6.0,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0,6.0,0.0,0.0,1.0,0.0,0%,15.0,3.0,15.0,3.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,1B,0.133333,,C+,12,26,0,0.0,,0.000,,0,n,8,8.0,22,0,0.0,,9,54,C+26 [n-22]
1,Glenn Adams,29,MIN,AL,95.0,290.0,269.0,32.0,91.0,17.0,0.0,6.0,49.0,0.0,2.0,18.0,30.0,0.338,0.376,0.468,0.844,130.0,126.0,5.0,0.0,0.0,3.0,3.0,D97H,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIN,AL,44.0,38.0,11.0,271.0,65.0,60.0,3.0,2.0,1.0,0.969,-3.0,-13.0,2.09,1.43,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.0,80.0,95.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,28.0,44.0,46.0,14.0,0.0,OF,0.515789,,A+,2,12,0,0.0,,0.000,,2,12,4,6.0,16,0,0.0,,11,52,A+12 [12-16]
2,Mike Adams,28,CHC,NL,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000,0.000,0.000,0.000,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,/87H,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHC,NL,2.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,-120.0,0.00,0.00,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,NaN,1.0,0.0,OF,0.000000,,G,0,,0,0.0,,0.000,,0,n,18,18.0,36,0,0.0,,0,66,G [n-36]
3,Willie Aikens,22,CAL,AL,42.0,101.0,91.0,5.0,18.0,4.0,0.0,0.0,6.0,1.0,2.0,10.0,23.0,0.198,0.277,0.242,0.519,46.0,22.0,1.0,0.0,0.0,0.0,2.0,H3D,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAL,AL,13.0,12.0,6.0,99.0,105.0,94.0,8.0,3.0,10.0,0.971,0.0,-2.0,9.27,7.85,1B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,23.0,42.0,13.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,18.0,0.0,1B,0.142857,,C,0,,0,0.0,,0.042,,4,14,8,12.0,26,0,0.0,,6,61,C [14-26]
4,Santo Alcala,24,TOT,NL,38.0,30.0,28.0,1.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,15.0,0.071,0.071,0.179,0.250,-34.0,5.0,0.0,0.0,2.0,0.0,0.0,1,R,NL,3.0,7.0,0.3,4.83,38.0,12.0,8.0,0.0,0.0,2.0,117.1,126.0,66.0,63.0,13.0,54.0,6.0,73.0,3.0,3.0,6.0,520.0,79.0,4.39,1.534,9.7,1.0,4.1,5.6,1.35,R,TOT,NL,38.0,12.0,0.0,117.1,19.0,8.0,11.0,0.0,0.0,1.000,NaN,NaN,1.46,0.50,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P,0.052632,,F,18,36,0,0.0,,0.000,,0,n,18,18.0,36,0,0.0,,2,65,F36 [n-36]


In [352]:
players.tail()

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos_Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating
1062,Mike Willis,26,TOR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,2.0,6.0,0.250,3.94,43.0,3.0,28.0,0.0,0.0,5.0,107.1,105.0,48.0,47.0,15.0,38.0,6.0,59.0,0.0,0.0,2.0,451.0,107.0,4.52,1.332,8.8,1.3,3.2,4.9,1.55,L,TOR,AL,43.0,3.0,0.0,107.1,26.0,6.0,19.0,1.0,1.0,0.962,NaN,NaN,2.10,0.58,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.0,3.0,0.0,43.0,43.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P,NaN,,G,0,,0,0.0,,0.0,,0,n,0,0.0,n,0,0.0,,0,66,G [n-n]
1063,Jim Willoughby,28,BOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,6.0,2.0,0.750,4.94,31.0,0.0,16.0,0.0,0.0,2.0,54.2,54.0,32.0,30.0,5.0,18.0,3.0,33.0,2.0,0.0,2.0,234.0,92.0,3.82,1.317,8.9,0.8,3.0,5.4,1.83,R,BOS,AL,31.0,0.0,0.0,54.2,23.0,12.0,10.0,1.0,1.0,0.957,NaN,NaN,3.62,0.71,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0,0.0,0.0,31.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P,NaN,,G,0,,0,0.0,,0.0,,0,n,0,0.0,n,0,0.0,,0,66,G [n-n]
1064,Rick Wise,31,BOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,11.0,5.0,0.688,4.77,26.0,20.0,2.0,4.0,2.0,0.0,128.1,151.0,68.0,68.0,19.0,28.0,1.0,85.0,4.0,1.0,1.0,555.0,95.0,4.09,1.395,10.6,1.3,2.0,6.0,3.04,R,BOS,AL,26.0,20.0,4.0,128.1,31.0,8.0,23.0,0.0,1.0,1.000,NaN,NaN,2.17,1.19,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0,20.0,0.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P,NaN,,G,0,,0,0.0,,0.0,,0,n,0,0.0,n,0,0.0,,0,66,G [n-n]
1065,Wilbur Wood,35,CHW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,7.0,8.0,0.467,4.99,24.0,18.0,2.0,5.0,1.0,0.0,122.2,139.0,75.0,68.0,10.0,50.0,0.0,42.0,10.0,0.0,0.0,548.0,83.0,4.58,1.541,10.2,0.7,3.7,3.1,0.84,L,CHW,AL,24.0,18.0,5.0,122.2,36.0,3.0,33.0,0.0,1.0,1.000,NaN,NaN,2.64,1.50,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,18.0,0.0,24.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P,NaN,,G,0,,0,0.0,,0.0,,0,n,0,0.0,n,0,0.0,,0,66,G [n-n]
1066,Geoff Zahn,31,MIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,12.0,14.0,0.462,4.68,34.0,32.0,0.0,7.0,1.0,0.0,198.0,234.0,116.0,103.0,20.0,66.0,4.0,88.0,5.0,0.0,5.0,870.0,85.0,4.24,1.515,10.6,0.9,3.0,4.0,1.33,L,MIN,AL,34.0,32.0,7.0,198.0,58.0,16.0,40.0,2.0,5.0,0.966,NaN,NaN,2.55,1.65,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0,32.0,0.0,34.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P,NaN,,G,0,,0,0.0,,0.0,,0,n,0,0.0,n,0,0.0,,0,66,G [n-n]


In [353]:
players

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos_Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating
0,Bob Adams,25,DET,AL,15.0,24.0,24.0,2.0,6.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,5.0,0.250,0.250,0.542,0.792,105.0,13.0,0.0,0.0,0.0,0.0,0.0,H/32,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DET,AL,3.0,3.0,3.0,27.0,27.0,26.0,1.0,0.0,1.0,1.000,-1.0,-22.0,9.00,9.00,1B-C,DET,AL,1.0,1.0,1.0,9.0,6.0,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0,6.0,0.0,0.0,1.0,0.0,0%,15.0,3.0,15.0,3.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,1B,0.133333,,C+,12,26,0,0.0,,0.000,,0,n,8,8.0,22,0,0.0,,9,54,C+26 [n-22]
1,Glenn Adams,29,MIN,AL,95.0,290.0,269.0,32.0,91.0,17.0,0.0,6.0,49.0,0.0,2.0,18.0,30.0,0.338,0.376,0.468,0.844,130.0,126.0,5.0,0.0,0.0,3.0,3.0,D97H,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIN,AL,44.0,38.0,11.0,271.0,65.0,60.0,3.0,2.0,1.0,0.969,-3.0,-13.0,2.09,1.43,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.0,80.0,95.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,28.0,44.0,46.0,14.0,0.0,OF,0.515789,,A+,2,12,0,0.0,,0.000,,2,12,4,6.0,16,0,0.0,,11,52,A+12 [12-16]
2,Mike Adams,28,CHC,NL,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000,0.000,0.000,0.000,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,/87H,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHC,NL,2.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,-120.0,0.00,0.00,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,NaN,1.0,0.0,OF,0.000000,,G,0,,0,0.0,,0.000,,0,n,18,18.0,36,0,0.0,,0,66,G [n-36]
3,Willie Aikens,22,CAL,AL,42.0,101.0,91.0,5.0,18.0,4.0,0.0,0.0,6.0,1.0,2.0,10.0,23.0,0.198,0.277,0.242,0.519,46.0,22.0,1.0,0.0,0.0,0.0,2.0,H3D,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAL,AL,13.0,12.0,6.0,99.0,105.0,94.0,8.0,3.0,10.0,0.971,0.0,-2.0,9.27,7.85,1B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,23.0,42.0,13.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,18.0,0.0,1B,0.142857,,C,0,,0,0.0,,0.042,,4,14,8,12.0,26,0,0.0,,6,61,C [14-26]
4,Santo Alcala,24,TOT,NL,38.0,30.0,28.0,1.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,15.0,0.071,0.071,0.179,0.250,-34.0,5.0,0.0,0.0,2.0,0.0,0.0,1,R,NL,3.0,7.0,0.300,4.83,38.0,12.0,8.0,0.0,0.0,2.0,117.1,126.0,66.0,63.0,13.0,54.0,6.0,73.0,3.0,3.0,6.0,520.0,79.0,4.39,1.534,9.7,1.0,4.1,5.6,1.35,R,TOT,NL,38.0,12.0,0.0,117.1,19.0,8.0,11.0,0.0,0.0,1.000,NaN,NaN,1.46,0.50,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P,0.052632,,F,18,36,0,0.0,,0.000,,0,n,18,18.0,36,0,0.0,,2,65,F36 [n-36]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [354]:
players.to_csv("../data/player stats - " + year + " - with batter ratings.csv", index=False)

## Pitcher Ratings

In [355]:
players = pd.read_csv("../data/player stats - " + year + " - with batter ratings.csv")

In [356]:
pd.set_option('display.max_seq_items', 150)
players.columns

Index(['Name', 'Age', 'Tm', 'Lg_bat', 'G_bat', 'PA', 'AB', 'R_bat', 'H_bat',
       '2B', '3B', 'HR_bat', 'RBI', 'SB', 'CS', 'BB_bat', 'SO_bat', 'BA',
       'OBP', 'SLG', 'OPS', 'OPS+', 'TB', 'GDP', 'HBP_bat', 'SH', 'SF',
       'IBB_bat', 'Pos_Summary', 'Bats', 'Lg_pit', 'W', 'L', 'W-L%', 'ERA',
       'G_pit', 'GS', 'GF', 'CG', 'SHO', 'SV', 'IP', 'H_pit', 'R_pit', 'ER',
       'HR_pit', 'BB_pit', 'IBB_pit', 'SO_pit', 'HBP_pit', 'BK', 'WP', 'BF',
       'ERA+', 'FIP', 'WHIP', 'H9', 'HR9', 'BB9', 'SO9', 'SO/W', 'Throws',
       'Tm_fld', 'Lg', 'G', 'GS_fld', 'CG_fld', 'Inn', 'Ch', 'PO', 'A', 'E',
       'DP', 'Fld%', 'Rtot', 'Rtot/yr', 'RF/9', 'RF/G', 'Pos_Summary_fld',
       'Tm_cat', 'Lg_cat', 'G_cat', 'GS_cat', 'CG_cat', 'Inn_cat', 'Ch_cat',
       'PO_cat', 'A_cat', 'E_cat', 'DP_cat', 'Fld%_cat', 'Rtot_cat',
       'Rtot/yr_cat', 'Rctch', 'RF/9_cat', 'RF/G_cat', 'PB', 'WP_cat',
       'SB_cat', 'CS_cat', 'CS%', 'G_app', 'GS_app', 'Batting', 'Defense', 'P',
       'C', '1B', '2B_a

### Pitcher Letter Rating

In [357]:
players["BAA"] = round(players["H_pit"] /(players["BF"] - (players["BB_pit"] + players["HBP_pit"])),3)
players["BAA"].mean()

0.27423799126637577

In [358]:
baa_break_points = [
    0.140,
    0.168,
    0.196,
    0.223,
    0.251,
    0.279,
    0.307,
    0.335,
    0.362
]

letters = [
    "J+",
    "J",
    "K",
    "L",
    "M",
    "W",
    "X",
    "Y",
    "Z+",
    "Z"
]

def pitcher_letter(bat_avg_against, breakpoints=baa_break_points, letter_grades=letters):
    i = bisect(breakpoints, bat_avg_against)
    return letter_grades[i]

In [359]:
players["pit_letter"] = [pitcher_letter(avg) for avg in players["BAA"]]
players["pit_letter"].value_counts()

Z     634
W     147
M     108
X      89
L      29
Y      27
Z+     15
K      12
J+      4
J       2
Name: pit_letter, dtype: int64

In [360]:
players.loc[(players["IP"].isnull()), "pit_letter"] = ""
players["pit_letter"].value_counts()

      609
W     147
M     108
X      89
L      29
Y      27
Z      25
Z+     15
K      12
J+      4
J       2
Name: pit_letter, dtype: int64

In [361]:
players[players["pit_letter"] == "K"]["BAA"].min()

0.174

### Innings of Effectiveness Number

**NOTE** - IP is stored as .0, .1, .2 for full, one third, and two-thirds, so these need to be set to their true decimal values before any calculations using IP can be done.

In [362]:
players["IP_real"] = round(players["IP"]) + (10 * (players["IP"] - round(players["IP"])) / 3)
players["IP_real"].value_counts().head(25)

21.000000     5
14.333333     5
6.333333      4
6.000000      4
7.000000      4
3.000000      3
106.333333    3
4.333333      3
8.333333      3
7.666667      3
72.666667     3
87.333333     3
27.333333     3
1.333333      3
108.000000    3
2.000000      3
20.666667     3
22.333333     3
20.333333     3
2.333333      3
5.666667      3
18.333333     3
2.666667      3
98.666667     3
151.000000    3
Name: IP_real, dtype: int64

In [363]:
players["IE"] = round(players["IP_real"] / players ["G_pit"], 0)
players["IE"].value_counts()

2.0    123
1.0     70
3.0     69
6.0     52
4.0     46
7.0     45
5.0     39
8.0     12
0.0      2
Name: IE, dtype: int64

In [364]:
pd.set_option('display.max_columns', 160)
players[players["IE"] == 0]

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos_Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE
917,Ed Farmer,27,BAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,0.0,0.0,NaN,inf,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,R,BAL,AL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P,NaN,NaN,G,0,NaN,0,0.0,NaN,0.0,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G [n-n],1.0,Z,0.0,0.0
965,Rick Kreuger,28,BOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,0.0,1.0,0.0,inf,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L,BOS,AL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P,NaN,NaN,G,0,NaN,0,0.0,NaN,0.0,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G [n-n],1.0,Z,0.0,0.0


In [365]:
# Reset 0 Innings of Effectiveness to 1 (can't have 0 in SherCo)
players["IE"].replace(0, 1, inplace=True)
players["IE"].value_counts()

2.0    123
1.0     72
3.0     69
6.0     52
4.0     46
7.0     45
5.0     39
8.0     12
Name: IE, dtype: int64

In [366]:
players[players["IE"] >= 7]

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos_Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE
68,Vida Blue,27,OAK,AL,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000,0.000,0.000,0.000,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,1,S,AL,14.0,19.0,0.424,3.83,38.0,38.0,0.0,16.0,1.0,0.0,279.2,284.0,138.0,119.0,23.0,86.0,5.0,157.0,1.0,0.0,11.0,1184.0,104.0,3.62,1.323,9.1,0.7,2.8,5.1,1.83,L,OAK,AL,38.0,38.0,16.0,279.2,51.0,6.0,42.0,3.0,2.0,0.941,NaN,NaN,1.54,1.26,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0,38.0,2.0,38.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.000,NaN,0,n,36,36.0,66,0,0.0,NaN,0,66,G [n-66],0.259,W,279.666667,7.0
89,Ken Brett,28,TOT,AL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1/H,L,AL,13.0,14.0,0.481,4.53,34.0,34.0,0.0,7.0,0.0,0.0,224.2,258.0,120.0,113.0,25.0,53.0,0.0,80.0,4.0,2.0,6.0,950.0,88.0,4.23,1.384,10.3,1.0,2.1,3.2,1.51,L,TOT,AL,34.0,34.0,7.0,224.2,61.0,11.0,48.0,2.0,6.0,0.967,NaN,NaN,2.36,1.74,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.000,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G [n-n],0.289,X,224.666667,7.0
90,Ken Brett,28,CAL,AL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1/H,L,AL,7.0,10.0,0.412,4.25,21.0,21.0,0.0,5.0,0.0,0.0,142.0,157.0,73.0,67.0,15.0,38.0,0.0,41.0,3.0,1.0,4.0,598.0,92.0,4.40,1.373,10.0,1.0,2.4,2.6,1.08,L,TOT,AL,34.0,34.0,7.0,224.2,61.0,11.0,48.0,2.0,6.0,0.967,NaN,NaN,2.36,1.74,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,21.0,1.0,21.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,P,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.000,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G [n-n],0.282,X,142.000000,7.0
115,John Candelaria,23,PIT,NL,33.0,88.0,80.0,5.0,18.0,3.0,2.0,0.0,11.0,0.0,1.0,4.0,18.0,0.225,0.262,0.313,0.574,53.0,25.0,0.0,0.0,4.0,0.0,0.0,1,L,NL,20.0,5.0,0.800,2.34,33.0,33.0,0.0,6.0,1.0,0.0,230.2,197.0,64.0,60.0,29.0,50.0,2.0,133.0,2.0,2.0,1.0,917.0,169.0,3.90,1.071,7.7,1.1,2.0,5.2,2.66,L,PIT,NL,33.0,33.0,6.0,230.2,37.0,6.0,30.0,1.0,3.0,0.973,NaN,NaN,1.40,1.09,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.0,33.0,33.0,33.0,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,P,0.333333,NaN,C+,0,NaN,4,4.0,(14),0.000,NaN,2,12,7,9.0,23,0,0.0,NaN,7,56,C+(14) [12-23],0.228,M,230.666667,7.0
124,Steve Carlton,32,PHI,NL,36.0,112.0,97.0,7.0,26.0,4.0,0.0,3.0,15.0,0.0,0.0,2.0,20.0,0.268,0.282,0.402,0.684,78.0,39.0,0.0,1.0,7.0,3.0,0.0,1,L,NL,23.0,10.0,0.697,2.64,36.0,36.0,0.0,17.0,2.0,0.0,283.0,229.0,99.0,83.0,25.0,89.0,5.0,198.0,4.0,7.0,3.0,1135.0,153.0,3.47,1.124,7.3,0.8,2.8,6.3,2.22,L,PHI,NL,36.0,36.0,17.0,283.0,57.0,4.0,52.0,1.0,2.0,0.982,NaN,NaN,1.78,1.56,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,36.0,36.0,36.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,P,0.416667,NaN,B,4,14.0,0,0.0,NaN,0.000,NaN,1,11,6,7.0,21,0,0.0,NaN,8,55,B14 [11-21],0.220,L,283.000000,8.0
142,Jim

In [367]:
players["IE"] = players["IE"].astype('Int64')

### Base on Balls Number

In [368]:
players["bb_rate"] = round(players["BB_pit"] / players["BF"] * 36, 0)
players["bb_rate"].replace(np.nan, 0, inplace=True)
players["bb_rate"] = players["bb_rate"].astype(int)
players["bb_rate"].value_counts()

0     620
3     177
4      90
2      89
5      47
1      11
6      10
8       9
7       8
9       2
18      1
12      1
11      1
10      1
Name: bb_rate, dtype: int64

In [369]:
players.loc[(players["BF"].isnull()), "bb_rate"] = np.nan
players["bb_rate"].value_counts()

3.0     177
4.0      90
2.0      89
5.0      47
1.0      11
0.0      11
6.0      10
8.0       9
7.0       8
9.0       2
11.0      1
18.0      1
12.0      1
10.0      1
Name: bb_rate, dtype: int64

In [370]:
players["bb_num_pit"] = players["bb_rate"].map({
    0: "11",
    1: "11",
    2: "12",
    3: "13",
    4: "14",
    5: "15",
    6: "16",
    7: "21",
    8: "22",
    9: "23",
    10: "24",
    11: "25",
    12: "26",
    13: "31",
    14: "32",
    15: "33",
    16: "34",
    17: "35",
    18: "36",
    19: "41",
    20: "42",
    21: "43",
    22: "44",
    23: "45",
    24: "46",
    25: "51",
    26: "52",
    27: "53",
    28: "54",
    29: "55",
    30: "56",
    31: "61",
    32: "62",
    33: "63",
    34: "64",
    35: "65",
    36: "66"
}).astype(str)
players["bb_num_pit"].value_counts()

nan    609
13     177
14      90
12      89
15      47
11      22
16      10
22       9
21       8
23       2
24       1
26       1
36       1
25       1
Name: bb_num_pit, dtype: int64

### Strikeout Number

In [371]:
players["k_rate_pit"] = round(players["SO_pit"] / players["BF"] * 36, 0)
players["k_rate_pit"].value_counts()

4.0     134
5.0     107
3.0      63
6.0      58
7.0      33
2.0      28
0.0      10
8.0       9
1.0       6
9.0       3
10.0      3
11.0      3
14.0      1
Name: k_rate_pit, dtype: int64

In [372]:
players[players["k_rate_pit"]==0]

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos_Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE,bb_rate,bb_num_pit,k_rate_pit
500,Larry McCall,24,NYY,AL,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,/1H,L,AL,0.0,1.0,0.0,7.50,2.0,0.0,0.0,0.0,0.0,0.0,6.0,12.0,7.0,5.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,33.0,56.0,5.41,2.167,18.0,1.5,1.5,0.0,0.0,R,NYY,AL,2.0,0.0,0.0,6.0,2.0,1.0,0.0,1.0,0.0,0.500,NaN,NaN,1.50,0.50,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,P,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.000,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G [n-n],0.375,Z,6.000000,3,1.0,11,0.0
510,Joey McLaughlin,20,ATL,NL,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000,0.000,0.000,0.000,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,/1,R,NL,0.0,0.0,NaN,15.00,3.0,2.0,1.0,0.0,0.0,0.0,6.0,10.0,10.0,10.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,31.0,31.0,10.74,2.167,15.0,4.5,4.5,0.0,0.0,R,ATL,NL,3.0,2.0,0.0,6.0,3.0,0.0,3.0,0.0,0.0,1.000,NaN,NaN,4.50,1.00,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,P,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.000,NaN,0,n,36,36.0,66,0,0.0,NaN,0,66,G [n-66],0.357,Z+,6.000000,2,3.0,13,0.0
520,Mario Mendoza,26,PIT,NL,70.0,86.0,81.0,5.0,16.0,3.0,0.0,0.0,4.0,0.0,0.0,3.0,10.0,0.198,0.226,0.235,0.461,23.0,19.0,1.0,0.0,2.0,0.0,0.0,65/H1,R,NL,0.0,0.0,NaN,13.50,1.0,0.0,1.0,0.0,0.0,0.0,2.0,3.0,3.0,3.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,10.0,35.0,12.24,2.500,13.5,4.5,9.0,0.0,0.0,R,PIT,NL,65.0,17.0,10.0,263.1,138.0,41.0,87.0,10.0,13.0,0.928,-1.0,-4.0,4.37,1.97,SS-3B-P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0,17.0,70.0,65.0,1.0,0.0,0.0,0.0,19.0,46.0,0.0,0.0,0.0,0.0,NaN,5.0,4.0,SS,0.057143,NaN,C,0,NaN,0,0.0,NaN,0.000,NaN,1,11,4,5.0,15,0,0.0,NaN,7,56,C [11-15],0.375,Z,2.000000,2,7.0,21,0.0
568,Jeff Newman,28,OAK,AL,94.0,172.0,162.0,17.0,36.0,9.0,0.0,4.0,15.0,2.0,0.0,4.0,24.0,0.222,0.244,0.352,0.596,63.0,57.0,2.0,1.0,4.0,1.0,1.0,2/1H,R,AL,0.0,0.0,NaN,0.00,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,NaN,5.74,1.000,9.0,0.0,0.0,0.0,NaN,R,OAK,AL,95.0,42.0,31.0,467.1,296.0,251.0,36.0,9.0,5.0,0.970,2.0,5.0,5.53,3.02,C-P,OAK,AL,94.0,42.0,31.0,466.1,296.0,251.0,36.0,9.0,5.0,0.97,2.0,5.0,2.0,5.54,3.05,4.0,20.0,27.0,29.0,52%,94.0,42.0,94.0,94.0,1.0,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,C,0.159574,NaN,C,4,14.0,0,0.0,NaN,0.071,NaN,1,11,5,6.0,16,0,0.0,NaN,8,55,C14 [11-16],0.250,M,1.000000,1,0.0,11,0.0
683,Ray Sadecki,36,NYM,NL,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,/1,L,NL,0.0,1.0,0.0,6.00,4.0,0.0,2.0,0.0,0.0,0.0,3.0,3.0,2.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,14.0,70.0,10.07,2.000,9.0,3.0,9.0,0.0,0.0,L,NYM,NL,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.00,0.00,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,0.0,4.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,P,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.000,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G [n-n],0.273,W,3.000000,1,8.0,22,0.0
917,Ed Farmer,27

In [373]:
players.loc[(players["k_rate_pit"] == 0), "k_val_pit"] = 0
players.loc[(players["k_rate_pit"] > 0), "k_val_pit"] = players["bb_rate"] + players["k_rate_pit"]
players["k_val_pit"].value_counts()

7.0     103
9.0      81
8.0      75
6.0      57
10.0     47
5.0      22
11.0     16
4.0      11
0.0      10
12.0      8
13.0      7
15.0      5
3.0       5
14.0      4
2.0       3
16.0      2
17.0      1
18.0      1
Name: k_val_pit, dtype: int64

In [374]:
players["k_num_pit"] = players["k_val_pit"].map({
    0: "n",
    1: "11",
    2: "12",
    3: "13",
    4: "14",
    5: "15",
    6: "16",
    7: "21",
    8: "22",
    9: "23",
    10: "24",
    11: "25",
    12: "26",
    13: "31",
    14: "32",
    15: "33",
    16: "34",
    17: "35",
    18: "36",
    19: "41",
    20: "42",
    21: "43",
    22: "44",
    23: "45",
    24: "46",
    25: "51",
    26: "52",
    27: "53",
    28: "54",
    29: "55",
    30: "56",
    31: "61",
    32: "62",
    33: "63",
    34: "64",
    35: "65",
    36: "66"
}).astype(str)
players["k_num_pit"].value_counts()

nan    609
21     103
23      81
22      75
16      57
24      47
15      22
25      16
14      11
n       10
26       8
31       7
33       5
13       5
32       4
12       3
34       2
36       1
35       1
Name: k_num_pit, dtype: int64

In [375]:
players[players["Name"] == "Dwight\xa0Gooden"]

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos_Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE,bb_rate,bb_num_pit,k_rate_pit,k_val_pit,k_num_pit


### Hit Batter Number

In [376]:
players["hbp_rate_pit"] = round(players["HBP_pit"] / players["BF"] * 36, 0)
players["hbp_rate_pit"].replace(np.nan, 0, inplace=True)
players["hbp_rate_pit"] = players["hbp_rate_pit"].astype(int)
players["hbp_rate_pit"].value_counts()

0    1035
1      26
3       3
2       2
7       1
Name: hbp_rate_pit, dtype: int64

In [377]:
players.loc[(players["hbp_rate_pit"] == 0), "hbp_val_pit"] = 0
players.loc[(players["hbp_rate_pit"] > 0), "hbp_val_pit"] = players["k_val_pit"] + players["hbp_rate_pit"]

In [378]:
players["hbp_val_pit"].value_counts()

0.0     1035
10.0       6
9.0        5
13.0       4
7.0        4
8.0        4
18.0       3
6.0        2
3.0        1
4.0        1
11.0       1
12.0       1
Name: hbp_val_pit, dtype: int64

In [379]:
players["hbp_num_pit"] = players["hbp_val_pit"].map({
    0: "",
    1: "/11",
    2: "/12",
    3: "/13",
    4: "/14",
    5: "/15",
    6: "/16",
    7: "/21",
    8: "/22",
    9: "/23",
    10: "/24",
    11: "/25",
    12: "/26",
    13: "/31",
    14: "/32",
    15: "/33",
    16: "/34",
    17: "/35",
    18: "/36",
    19: "/41",
    20: "/42",
    21: "/43",
    22: "/44",
    23: "/45",
    24: "/46",
    25: "/51",
    26: "/52",
    27: "/53",
    28: "/54",
    29: "/55",
    30: "/56",
    31: "/61",
    32: "/62",
    33: "/63",
    34: "/64",
    35: "/65",
    36: "/66"
}).astype(str)
players["hbp_num_pit"].value_counts()

       1035
/24       6
/23       5
/22       4
/21       4
/31       4
/36       3
/16       2
/13       1
/26       1
/14       1
/25       1
Name: hbp_num_pit, dtype: int64

### Wild Pitch Rating

In [380]:
pd.set_option('display.max_seq_items', 200)
players.columns

Index(['Name', 'Age', 'Tm', 'Lg_bat', 'G_bat', 'PA', 'AB', 'R_bat', 'H_bat',
       '2B', '3B', 'HR_bat', 'RBI', 'SB', 'CS', 'BB_bat', 'SO_bat', 'BA',
       'OBP', 'SLG', 'OPS', 'OPS+', 'TB', 'GDP', 'HBP_bat', 'SH', 'SF',
       'IBB_bat', 'Pos_Summary', 'Bats', 'Lg_pit', 'W', 'L', 'W-L%', 'ERA',
       'G_pit', 'GS', 'GF', 'CG', 'SHO', 'SV', 'IP', 'H_pit', 'R_pit', 'ER',
       'HR_pit', 'BB_pit', 'IBB_pit', 'SO_pit', 'HBP_pit', 'BK', 'WP', 'BF',
       'ERA+', 'FIP', 'WHIP', 'H9', 'HR9', 'BB9', 'SO9', 'SO/W', 'Throws',
       'Tm_fld', 'Lg', 'G', 'GS_fld', 'CG_fld', 'Inn', 'Ch', 'PO', 'A', 'E',
       'DP', 'Fld%', 'Rtot', 'Rtot/yr', 'RF/9', 'RF/G', 'Pos_Summary_fld',
       'Tm_cat', 'Lg_cat', 'G_cat', 'GS_cat', 'CG_cat', 'Inn_cat', 'Ch_cat',
       'PO_cat', 'A_cat', 'E_cat', 'DP_cat', 'Fld%_cat', 'Rtot_cat',
       'Rtot/yr_cat', 'Rctch', 'RF/9_cat', 'RF/G_cat', 'PB', 'WP_cat',
       'SB_cat', 'CS_cat', 'CS%', 'G_app', 'GS_app', 'Batting', 'Defense', 'P',
       'C', '1B', '2B_a

In [381]:
players["WP"].value_counts()

0.0     105
1.0      75
2.0      75
3.0      57
4.0      48
5.0      28
6.0      22
8.0      16
7.0      11
9.0       5
11.0      4
10.0      4
12.0      3
14.0      2
20.0      1
17.0      1
21.0      1
Name: WP, dtype: int64

In [382]:
players[players["WP"] >= 10]

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos_Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE,bb_rate,bb_num_pit,k_rate_pit,k_val_pit,k_num_pit,hbp_rate_pit,hbp_val_pit,hbp_num_pit
68,Vida Blue,27,OAK,AL,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000,0.000,0.000,0.000,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,1,S,AL,14.0,19.0,0.424,3.83,38.0,38.0,0.0,16.0,1.0,0.0,279.2,284.0,138.0,119.0,23.0,86.0,5.0,157.0,1.0,0.0,11.0,1184.0,104.0,3.62,1.323,9.1,0.7,2.8,5.1,1.83,L,OAK,AL,38.0,38.0,16.0,279.2,51.0,6.0,42.0,3.0,2.0,0.941,NaN,NaN,1.54,1.26,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0,38.0,2.0,38.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.0,NaN,0,n,36,36.0,66,0,0.0,NaN,0,66,G [n-66],0.259,W,279.666667,7,3.0,13,5.0,8.0,22,0,0.0,
385,Tom Johnson,26,MIN,AL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1,R,AL,16.0,7.0,0.696,3.13,71.0,0.0,54.0,0.0,0.0,15.0,146.2,152.0,57.0,51.0,11.0,47.0,8.0,87.0,5.0,0.0,10.0,629.0,128.0,3.59,1.357,9.3,0.7,2.9,5.3,1.85,R,MIN,AL,71.0,0.0,0.0,146.2,36.0,9.0,26.0,1.0,0.0,0.972,NaN,NaN,2.15,0.49,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.0,0.0,1.0,71.0,71.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.0,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G [n-n],0.263,W,146.666667,2,3.0,13,5.0,8.0,22,0,0.0,
450,Dennis Leonard,26,KCR,AL,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1/D,R,AL,20.0,12.0,0.625,3.04,38.0,37.0,1.0,21.0,5.0,1.0,292.2,246.0,117.0,99.0,18.0,79.0,0.0,244.0,8.0,2.0,14.0,1186.0,134.0,2.76,1.110,7.6,0.6,2.4,7.5,3.09,R,KCR,AL,38.0,37.0,21.0,292.2,54.0,21.0,29.0,4.0,2.0,0.926,NaN,NaN,1.54,1.32,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,38.0,2.0,38.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,P,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.0,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G [n-n],0.224,M,292.666667,8,2.0,12,7.0,9.0,23,0,0.0,
467,Sparky Lyle,32,NYY,AL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1,L,AL,13.0,5.0,0.722,2.17,72.0,0.0,60.0,0.0,0.0,26.0,137.0,131.0,41.0,33.0,7.0,33.0,6.0,68.0,2.0,0.0,10.0,554.0,183.0,3.18,1.197,8.6,0.5,2.2,4.5,2.06,L,NYY,AL,72.0,0.0,0.0,137.0,26.0,2.0,22.0,2.0,0.0,0.923,NaN,NaN,1.58,0.33,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.0,0.0,1.0,72.0,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.0,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G [n-n],0.252,W,137.000000,2,2.0,12,4.0,6.0,16,0,0.0,
570,Phil Niekro,38,ATL,NL,44.0,123.0,109.0,2.0,19.0,1.0,0.0,0.0,7.0,0.0,0.0,0.0,15.0,0.174,0.180,0.183,0.364,-6.0,20.0,3.0,1.0,12.0,1.0,0.0,1,R,NL,16.0,20.0,0.444,4.03,44.0,43.0,1.0,20.0,2.0,0.0,330.1,315.0,166.0,148.0,26.0,164.0,12.0,262.0,8.0,3.0,17.0,1428.0,111.0,3.74,1.450,8.6,0.7,4.5,7.1,1.60,R,ATL,NL,44.0,43.0,20.0,330.1,71.0,20.0,51.0,0.0,5.0,1.000,NaN,NaN,1.93,1.61,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.0,43.0,44.0,44.0,44.0,0.0,0

In [383]:
players.loc[(players["WP"] < 5), "WP_num"] = ""
players.loc[(players["WP"] >= 5), "WP_num"] = "[WP]"
players["WP_num"].value_counts()

        360
[WP]     98
Name: WP_num, dtype: int64

### Gopher Ball Rating

In [384]:
players["hr_rate_pit"] = players["HR_pit"] / players["H_pit"]
players["hr_rate_pit"].value_counts()

0.000000    39
0.142857     8
0.200000     7
0.333333     6
0.083333     6
0.100000     5
0.125000     5
0.090909     5
0.055556     5
0.086957     3
0.105263     3
0.185185     3
0.045455     3
0.102564     3
0.115385     3
0.074074     3
0.166667     3
0.061224     3
0.073171     3
0.181818     3
0.034483     3
0.066667     3
0.052632     3
0.111111     3
0.250000     3
0.095238     3
0.214286     3
0.038462     3
0.133333     3
0.150943     2
0.103175     2
0.054545     2
0.050000     2
0.123457     2
0.058824     2
0.094340     2
0.110497     2
0.130000     2
0.040000     2
0.096774     2
0.098765     2
0.071429     2
0.094787     2
0.123810     2
0.025000     2
0.092593     2
0.139860     2
0.222222     2
0.097561     2
0.127660     2
0.053435     2
0.106383     2
0.088608     2
0.078947     2
0.060000     2
0.140351     2
0.060606     2
0.095541     2
0.140000     2
0.285714     2
0.072464     2
0.082051     2
0.117647     2
0.150000     2
0.068966     2
0.109375     2
0.089744  

In [385]:
players["gopher_ball"] = ""
players.loc[(players["hr_rate_pit"] >= .1), "gopher_ball"] = "+"
players.loc[(players["hr_rate_pit"] <= .05), "gopher_ball"] = "-"
players["gopher_ball"].value_counts()

     781
+    209
-     77
Name: gopher_ball, dtype: int64

In [386]:
players[players["gopher_ball"] == "-"]

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos_Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE,bb_rate,bb_num_pit,k_rate_pit,k_val_pit,k_num_pit,hbp_rate_pit,hbp_val_pit,hbp_num_pit,WP_num,hr_rate_pit,gopher_ball
5,Santo Alcala,24,CIN,NL,7.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.000,0.000,0.000,0.000,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,/1,R,NL,1.0,1.0,0.500,5.74,7.0,2.0,2.0,0.0,0.0,0.0,15.2,22.0,11.0,10.0,1.0,7.0,1.0,9.0,1.0,1.0,2.0,73.0,70.0,3.95,1.851,12.6,0.6,4.0,5.2,1.29,R,TOT,NL,38.0,12.0,0.0,117.1,19.0,8.0,11.0,0.0,0.0,1.000,NaN,NaN,1.46,0.50,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,2.0,7.0,7.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,P,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.000,NaN,0,n,24,24.0,46,0,0.0,NaN,0,66,G [n-46],0.338,Z+,15.666667,2,3.0,13,4.0,7.0,21,0,0.0,,,0.045455,-
111,Mike Caldwell,28,CIN,NL,14.0,5.0,4.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.500,0.600,1.000,1.600,318.0,4.0,0.0,0.0,0.0,0.0,0.0,1,R,NL,0.0,0.0,NaN,4.01,14.0,0.0,5.0,0.0,0.0,1.0,24.2,25.0,11.0,11.0,1.0,8.0,1.0,11.0,0.0,0.0,1.0,105.0,99.0,3.35,1.338,9.1,0.4,2.9,4.0,1.38,L,TOT,ZZ,35.0,12.0,2.0,119.0,38.0,7.0,31.0,0.0,3.0,1.000,NaN,NaN,2.87,1.09,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,0.0,14.0,14.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,P,0.000000,NaN,AAA,0,NaN,0,0.0,NaN,0.000,NaN,7,21,0,0.0,n,0,0.0,NaN,14,45,AAA [21-n],0.258,W,24.666667,2,3.0,13,4.0,7.0,21,0,0.0,,,0.040000,-
118,Doug Capilla,25,STL,NL,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,/1,L,NL,0.0,0.0,NaN,15.43,2.0,0.0,0.0,0.0,0.0,0.0,2.1,2.0,4.0,4.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,11.0,29.0,4.45,1.714,7.7,0.0,7.7,3.9,0.50,L,TOT,NL,24.0,16.0,1.0,108.2,22.0,6.0,13.0,3.0,0.0,0.864,NaN,NaN,1.57,0.79,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,P,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.000,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G [n-n],0.222,L,2.333333,1,7.0,21,3.0,10.0,24,0,0.0,,,0.000000,-
170,John D'Acquisto,25,STL,NL,3.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000,0.000,0.000,0.000,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,/1,R,NL,0.0,0.0,NaN,4.32,3.0,2.0,1.0,0.0,0.0,0.0,8.1,5.0,4.0,4.0,0.0,10.0,0.0,9.0,1.0,0.0,2.0,39.0,94.0,4.54,1.800,5.4,0.0,10.8,9.7,0.90,R,TOT,NL,20.0,14.0,0.0,52.1,12.0,4.0,6.0,2.0,3.0,0.833,NaN,NaN,1.72,0.50,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,P,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.000,NaN,0,n,18,18.0,36,0,0.0,NaN,0,66,G [n-36],0.179,K,8.333333,3,9.0,23,8.0,17.0,35,1,18.0,/36,,0.000000,-
196,Tom Dixon,22,HOU,NL,9.0,10.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.000,0.000,0.000,0.000,-100.0,0.0,0.0,0.0,3.0,0.0,0.0,/1,R,NL,1.0,0.0,1.000,3.26,9.0,4.0,0.0,1.0,0.0,0.0,30.1,40.0,12.0,11.0,0.0,7.0,0.0,15.0,1.0,1.0,1.0,135.0,111.0,2.54,1.549,11.9,0.0,2.1,4.5,2.14,R,HOU,NL,9.0,4.0,1.0,30.1,10.0,2.0,6.0,2.0,0.0,0.800,NaN,NaN,2.37,0.89,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,4.0,9.

### Pitcher Control Number

In [387]:
players["control_rate"] = round((players["BB_pit"] + players["HBP_pit"] + players["H_pit"]) / 
                                players["BF"] * 36, 0)
players["control_rate"].value_counts()

12.0    120
11.0    106
13.0     83
14.0     50
10.0     37
15.0     15
17.0     11
9.0       9
16.0      9
18.0      4
8.0       3
24.0      2
22.0      2
20.0      2
36.0      2
7.0       1
4.0       1
21.0      1
Name: control_rate, dtype: int64

In [388]:
players["PCN"] = players["control_rate"].map({
    0: "65",
    1: "65",
    2: "64",
    3: "63",
    4: "62",
    5: "61",
    6: "56",
    7: "55",
    8: "54",
    9: "53",
    10: "52",
    11: "51",
    12: "46",
    13: "45",
    14: "44",
    15: "43",
    16: "42",
    17: "41",
    18: "36",
    19: "35",
    20: "34",
    21: "33",
    22: "32",
    23: "31",
    24: "26",
    25: "25",
    26: "24",
    27: "23",
    28: "22",
    29: "21",
    30: "16",
    31: "15",
    32: "14",
    33: "13",
    34: "12",
    35: "11",
    36: "11"
}).astype(str)
players["PCN"].value_counts()

nan    609
46     120
51     106
45      83
44      50
52      37
43      15
41      11
42       9
53       9
36       4
54       3
26       2
32       2
11       2
34       2
33       1
55       1
62       1
Name: PCN, dtype: int64

### Probable Hit Number

In [389]:
players["hit_rate_pit"] = round(players["H_pit"] / players["BF"] * 36, 0)
players["hit_rate_pit"].replace(np.nan, 0, inplace=True)
players["hit_rate_pit"] = players["hit_rate_pit"].astype(int)
players["hit_rate_pit"].value_counts()

0     609
8     137
9     126
10     66
7      57
11     23
6      15
12      8
14      5
13      5
4       5
15      2
18      2
5       2
24      1
20      1
22      1
2       1
36      1
Name: hit_rate_pit, dtype: int64

In [390]:
players.loc[(players["BF"].isnull()), "hit_rate_pit"] = np.nan

In [391]:
players["hit_rate_pit"].value_counts()

8.0     137
9.0     126
10.0     66
7.0      57
11.0     23
6.0      15
12.0      8
14.0      5
4.0       5
13.0      5
5.0       2
18.0      2
15.0      2
24.0      1
20.0      1
36.0      1
22.0      1
2.0       1
Name: hit_rate_pit, dtype: int64

In [392]:
players["PPH"] = players["hit_rate_pit"].map({
    0: "66",
    1: "66",
    2: "65",
    3: "64",
    4: "63",
    5: "62",
    6: "61",
    7: "56",
    8: "55",
    9: "54",
    10: "53",
    11: "52",
    12: "51",
    13: "46",
    14: "45",
    15: "44",
    16: "43",
    17: "42",
    18: "41",
    19: "36",
    20: "35",
    21: "34",
    22: "33",
    23: "32",
    24: "31",
    25: "26",
    26: "25",
    27: "24",
    28: "23",
    29: "22",
    30: "21",
    31: "16",
    32: "15",
    33: "14",
    34: "13",
    35: "12",
    36: "11"
}).astype(str)
players["PPH"].value_counts()

nan    609
55     137
54     126
53      66
56      57
52      23
61      15
51       8
46       5
63       5
45       5
44       2
62       2
41       2
33       1
31       1
65       1
35       1
11       1
Name: PPH, dtype: int64

### Pitcher Rating

In [393]:
players.columns

Index(['Name', 'Age', 'Tm', 'Lg_bat', 'G_bat', 'PA', 'AB', 'R_bat', 'H_bat',
       '2B', '3B', 'HR_bat', 'RBI', 'SB', 'CS', 'BB_bat', 'SO_bat', 'BA',
       'OBP', 'SLG', 'OPS', 'OPS+', 'TB', 'GDP', 'HBP_bat', 'SH', 'SF',
       'IBB_bat', 'Pos_Summary', 'Bats', 'Lg_pit', 'W', 'L', 'W-L%', 'ERA',
       'G_pit', 'GS', 'GF', 'CG', 'SHO', 'SV', 'IP', 'H_pit', 'R_pit', 'ER',
       'HR_pit', 'BB_pit', 'IBB_pit', 'SO_pit', 'HBP_pit', 'BK', 'WP', 'BF',
       'ERA+', 'FIP', 'WHIP', 'H9', 'HR9', 'BB9', 'SO9', 'SO/W', 'Throws',
       'Tm_fld', 'Lg', 'G', 'GS_fld', 'CG_fld', 'Inn', 'Ch', 'PO', 'A', 'E',
       'DP', 'Fld%', 'Rtot', 'Rtot/yr', 'RF/9', 'RF/G', 'Pos_Summary_fld',
       'Tm_cat', 'Lg_cat', 'G_cat', 'GS_cat', 'CG_cat', 'Inn_cat', 'Ch_cat',
       'PO_cat', 'A_cat', 'E_cat', 'DP_cat', 'Fld%_cat', 'Rtot_cat',
       'Rtot/yr_cat', 'Rctch', 'RF/9_cat', 'RF/G_cat', 'PB', 'WP_cat',
       'SB_cat', 'CS_cat', 'CS%', 'G_app', 'GS_app', 'Batting', 'Defense', 'P',
       'C', '1B', '2B_a

In [394]:
players["goph_lett_inn"] = players["gopher_ball"] + players["pit_letter"] + players["IE"].astype(str)
players.loc[(players["IP"].isnull()), "goph_lett_inn"] = ""
players["goph_lett_inn"].value_counts()

        609
+M2      22
+W6      15
W6       15
W2       13
W7       13
+W2      13
X5       12
+M7      11
+W4      10
+M3      10
+W3      10
+X2       9
+Z1       8
M6        8
+W5       8
X4        8
+X4       7
X2        7
-M2       7
+X1       7
-M1       7
M2        7
+W7       6
-Z1       6
W4        6
X3        6
X1        6
W1        6
W3        5
-X1       5
+L2       5
W5        5
X6        5
M8        5
+W1       5
L2        5
+M5       5
M7        5
L7        4
+X3       4
+Z+2      4
+Y2       4
-Y3       4
-W2       4
-L3       4
M1        4
+M6       4
+M1       3
-W1       3
+Y4       3
+L8       3
+K2       3
-W3       3
+Z2       3
M3        3
Y3        3
-K3       3
Z2        3
-X3       3
+X5       3
+M4       3
-L2       2
-J+3      2
+L1       2
+Z+3      2
Y2        2
M4        2
-Y2       2
+Z+4      2
+K1       2
Y5        2
-W6       2
+Z3       2
-Z+2      2
-W7       2
X7        2
+X6       2
K2        2
-M7       1
-X5       1
+Y5       1
L8        1
+W8 

In [395]:
players["bb_k_hbp"] = "(" + players["bb_num_pit"] + "-" + players["k_num_pit"] + players["hbp_num_pit"] + ") "
players["bb_k_hbp"].value_counts()

(nan-nan)      609
(13-21)         64
(13-22)         34
(13-23)         26
(14-23)         25
(12-16)         25
(13-16)         21
(14-22)         18
(15-24)         17
(12-21)         17
(14-21)         15
(13-24)         12
(12-22)         12
(12-15)         12
(14-24)         10
(12-23)         10
(15-23)          9
(14-25)          6
(12-14)          6
(15-25)          6
(14-16)          5
(13-15)          5
(15-22)          4
(11-14)          4
(13-21/22)       3
(14-23/24)       3
(11-15)          3
(11-n)           3
(22-24)          3
(21-n)           3
(16-23)          3
(14-22/23)       2
(15-26)          2
(22-31)          2
(12-16/21)       2
(21-24)          2
(13-31)          2
(11-13)          2
(11-16)          2
(16-25/31)       2
(11-12)          2
(13-22/23)       2
(14-26)          2
(11-21)          2
(15-23/24)       2
(16-22)          2
(12-13)          2
(22-n)           1
(11-13/14)       1
(13-16/21)       1
(21-23)          1
(12-15/16)       1
(21-32)     

In [396]:
players.loc[(players["IP"].isnull()), "bb_k_hbp"] = ""
players["bb_k_hbp"].value_counts()

               609
(13-21)         64
(13-22)         34
(13-23)         26
(12-16)         25
(14-23)         25
(13-16)         21
(14-22)         18
(15-24)         17
(12-21)         17
(14-21)         15
(12-22)         12
(13-24)         12
(12-15)         12
(12-23)         10
(14-24)         10
(15-23)          9
(14-25)          6
(15-25)          6
(12-14)          6
(14-16)          5
(13-15)          5
(11-14)          4
(15-22)          4
(22-24)          3
(11-n)           3
(16-23)          3
(13-21/22)       3
(11-15)          3
(14-23/24)       3
(21-n)           3
(15-26)          2
(12-16/21)       2
(11-13)          2
(13-31)          2
(14-22/23)       2
(21-24)          2
(22-31)          2
(11-16)          2
(16-25/31)       2
(11-12)          2
(11-21)          2
(13-22/23)       2
(15-23/24)       2
(16-22)          2
(12-13)          2
(14-26)          2
(22-34)          1
(21-32)          1
(14-31)          1
(11-13/14)       1
(12-31)          1
(13-16/21)  

In [397]:
players["pitcher_rating"] = players["goph_lett_inn"] + " " + players["bb_k_hbp"] + " " + players["WP_num"]
players["pitcher_rating"].value_counts()

+W6 (13-21)            5
+M2 (14-22)            4
+W7 (13-21)            4
+M2 (13-22)            3
W6 (13-21)  [WP]       3
L2 (13-24)             2
Y3 (13-21)             2
M7 (12-21)             2
X4 (14-23)  [WP]       2
W2 (13-22)  [WP]       2
+M7 (13-16)            2
+L2 (14-25)            2
+M2 (12-15)            2
+W4 (12-16)            2
W2 (13-22)             2
W7 (13-21)             2
X3 (13-21)             2
+W6 (13-16)            2
M2 (12-23)             2
+M2 (15-25)  [WP]      2
L7 (13-24)  [WP]       2
W7 (13-21)  [WP]       2
-Y3 (13-21)            2
W2 (14-23)  [WP]       2
M8 (13-22)  [WP]       2
M4 (13-22)  [WP]       2
W6 (13-21)             2
+X1 (14-16)            2
-Z1 (11-n)             2
+M5 (15-25)  [WP]      2
L2 (13-23)             2
W4 (13-16)             2
-X3 (13-21)            2
W7 (13-22)  [WP]       2
W6 (12-15)             2
X5 (12-14)             2
M1 (13-22)             2
+M3 (22-31)            2
+W3 (14-23)            2
M6 (13-23)  [WP]       2


In [398]:
players[players["Name"] == "Dwight\xa0Gooden"]

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos_Summary_fld,Tm_cat,...,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE,bb_rate,bb_num_pit,k_rate_pit,k_val_pit,k_num_pit,hbp_rate_pit,hbp_val_pit,hbp_num_pit,WP_num,hr_rate_pit,gopher_ball,control_rate,PCN,hit_rate_pit,PPH,goph_lett_inn,bb_k_hbp,pitcher_rating


In [399]:
players.head(10)

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos_Summary_fld,Tm_cat,...,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE,bb_rate,bb_num_pit,k_rate_pit,k_val_pit,k_num_pit,hbp_rate_pit,hbp_val_pit,hbp_num_pit,WP_num,hr_rate_pit,gopher_ball,control_rate,PCN,hit_rate_pit,PPH,goph_lett_inn,bb_k_hbp,pitcher_rating
0,Bob Adams,25,DET,AL,15.0,24.0,24.0,2.0,6.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,5.0,0.250,0.250,0.542,0.792,105.0,13.0,0.0,0.0,0.0,0.0,0.0,H/32,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DET,AL,3.0,3.0,3.0,27.0,27.0,26.0,1.0,0.0,1.0,1.000,-1.0,-22.0,9.00,9.00,1B-C,DET,...,1.0,1.0,9.0,6.0,6.0,0.0,0.0,0.0,1.000,0.0,0.0,0.0,6.0,6.00,0.0,0.0,1.0,0.0,0%,15.0,3.0,15.0,3.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,1B,0.133333,NaN,C+,12,26.0,0,0.0,NaN,0.000,NaN,0,n,8,8.0,22,0,0.0,NaN,9,54,C+26 [n-22],NaN,,NaN,<NA>,NaN,nan,NaN,NaN,nan,0,0.0,,NaN,NaN,,NaN,nan,NaN,nan,,,NaN
1,Glenn Adams,29,MIN,AL,95.0,290.0,269.0,32.0,91.0,17.0,0.0,6.0,49.0,0.0,2.0,18.0,30.0,0.338,0.376,0.468,0.844,130.0,126.0,5.0,0.0,0.0,3.0,3.0,D97H,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIN,AL,44.0,38.0,11.0,271.0,65.0,60.0,3.0,2.0,1.0,0.969,-3.0,-13.0,2.09,1.43,OF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.0,80.0,95.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,28.0,44.0,46.0,14.0,0.0,OF,0.515789,NaN,A+,2,12.0,0,0.0,NaN,0.000,NaN,2,12,4,6.0,16,0,0.0,NaN,11,52,A+12 [12-16],NaN,,NaN,<NA>,NaN,nan,NaN,NaN,nan,0,0.0,,NaN,NaN,,NaN,nan,NaN,nan,,,NaN
2,Mike Adams,28,CHC,NL,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000,0.000,0.000,0.000,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,/87H,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHC,NL,2.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,-120.0,0.00,0.00,OF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,NaN,1.0,0.0,OF,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.000,NaN,0,n,18,18.0,36,0,0.0,NaN,0,66,G [n-36],NaN,,NaN,<NA>,NaN,nan,NaN,NaN,nan,0,0.0,,NaN,NaN,,NaN,nan,NaN,nan,,,NaN
3,Willie Aikens,22,CAL,AL,42.0,101.0,91.0,5.0,18.0,4.0,0.0,0.0,6.0,1.0,2.0,10.0,23.0,0.198,0.277,0.242,0.519,46.0,22.0,1.0,0.0,0.0,0.0,2.0,H3D,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAL,AL,13.0,12.0,6.0,99.0,105.0,94.0,8.0,3.0,10.0,0.971,0.0,-2.0,9.27,7.85,1B,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,23.0,42.0,13.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,18.0,0.0,1B,0.142857,NaN,C,0,NaN,0,0.0,NaN,0.042,NaN,4,14,8,12.0,26,0,0.0,NaN,6,61,C [14-26],NaN,,NaN,<NA>,NaN,nan,NaN,NaN,nan,0,0.0,,NaN,NaN,,NaN,nan,NaN,nan,,,NaN
4,Santo Alcala,24,TOT,NL,38.0,30.0,28.0,1.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,15.0,0.071,0.071,0.179,0.250,-34.0,5.0,0.0,0.0,2.0,0.0,0.0,1,R,NL,3.0,7.0,0.30,4.83,38.0,12.0,8.0,0.0,0.0,2.0,117.1,126.0,66.0,63.0,13.0,54.0,6.0,73.0,3.0,3.0,6.0,520.0,79.0,4.39,1.534,9.7,1.0,4.1,5.6,1.35,R,TOT,NL,38.0,12.0,0.0,117.1,19.

In [400]:
players.tail(10)

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos_Summary_fld,Tm_cat,...,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE,bb_rate,bb_num_pit,k_rate_pit,k_val_pit,k_num_pit,hbp_rate_pit,hbp_val_pit,hbp_num_pit,WP_num,hr_rate_pit,gopher_ball,control_rate,PCN,hit_rate_pit,PPH,goph_lett_inn,bb_k_hbp,pitcher_rating
1057,Tom Walker,28,CAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,0.0,0.0,NaN,9.00,1.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,2.0,2.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,8.0,52.0,14.74,1.500,13.5,9.0,0.0,4.5,NaN,R,TOT,ZZ,12.0,0.0,0.0,21.0,3.0,1.0,2.0,0.0,0.0,1.000,NaN,NaN,1.29,0.25,P,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P,NaN,NaN,G,0,NaN,0,0.0,NaN,0.0,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G [n-n],0.375,Z,2.000000,2,0.0,11,4.0,4.0,14,0,0.0,,,0.666667,+,14.0,44,14.0,45,+Z2,(11-14),+Z2 (11-14)
1058,Mike Wallace,26,TEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,0.0,0.0,NaN,7.56,5.0,0.0,0.0,0.0,0.0,0.0,8.1,10.0,7.0,7.0,1.0,10.0,0.0,2.0,0.0,0.0,1.0,43.0,57.0,7.42,2.400,10.8,1.1,10.8,2.2,0.20,L,TEX,AL,5.0,0.0,0.0,8.1,3.0,0.0,3.0,0.0,0.0,1.000,NaN,NaN,3.24,0.60,P,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P,NaN,NaN,G,0,NaN,0,0.0,NaN,0.0,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G [n-n],0.303,X,8.333333,2,8.0,22,2.0,10.0,24,0,0.0,,,0.100000,+,17.0,41,8.0,55,+X2,(22-24),+X2 (22-24)
1059,Gary Wheelock,25,SEA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,6.0,9.0,0.400,4.89,17.0,17.0,0.0,2.0,0.0,0.0,88.1,94.0,58.0,48.0,16.0,26.0,0.0,47.0,2.0,0.0,3.0,382.0,84.0,4.98,1.358,9.6,1.6,2.6,4.8,1.81,R,SEA,AL,17.0,17.0,2.0,88.1,21.0,10.0,9.0,2.0,0.0,0.905,NaN,NaN,1.94,1.12,P,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,17.0,0.0,17.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P,NaN,NaN,G,0,NaN,0,0.0,NaN,0.0,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G [n-n],0.266,W,88.333333,5,2.0,12,4.0,6.0,16,0,0.0,,,0.170213,+,11.0,51,9.0,54,+W5,(12-16),+W5 (12-16)
1060,Milt Wilcox,27,DET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,6.0,2.0,0.750,3.64,20.0,13.0,4.0,1.0,0.0,0.0,106.1,96.0,46.0,43.0,13.0,37.0,1.0,82.0,1.0,1.0,1.0,441.0,118.0,3.86,1.251,8.1,1.1,3.1,6.9,2.22,R,DET,AL,20.0,13.0,1.0,106.1,20.0,4.0,14.0,2.0,2.0,0.900,NaN,NaN,1.52,0.90,P,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,13.0,0.0,20.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P,NaN,NaN,G,0,NaN,0,0.0,NaN,0.0,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G [n-n],0.238,M,106.333333,5,3.0,13,7.0,10.0,24,0,0.0,,,0.135417,+,11.0,51,8.0,55,+M5,(13-24),+M5 (13-24)
1061,Randy Wiles,25,CHW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,1.0,1.0,0.500,10.13,5.0,0.0,4.0,0.0,0.0,0.0,2.2,5.0,3.0,3.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,16.0,46.0,10.99,3.000,16.9,3.4,10.1,0.0,0.00,L,CHW,AL,5.0,0.0,0.0,2.2,1.0,0.0,

In [401]:
players.to_csv("../data/player stats - " + year + " - with batter and pitcher ratings.csv", index=False)

## Fielding Ratings

In [402]:
players = pd.read_csv("../data/player stats - " + year + " - with batter and pitcher ratings.csv")

In [403]:
players.columns

Index(['Name', 'Age', 'Tm', 'Lg_bat', 'G_bat', 'PA', 'AB', 'R_bat', 'H_bat',
       '2B', '3B', 'HR_bat', 'RBI', 'SB', 'CS', 'BB_bat', 'SO_bat', 'BA',
       'OBP', 'SLG', 'OPS', 'OPS+', 'TB', 'GDP', 'HBP_bat', 'SH', 'SF',
       'IBB_bat', 'Pos_Summary', 'Bats', 'Lg_pit', 'W', 'L', 'W-L%', 'ERA',
       'G_pit', 'GS', 'GF', 'CG', 'SHO', 'SV', 'IP', 'H_pit', 'R_pit', 'ER',
       'HR_pit', 'BB_pit', 'IBB_pit', 'SO_pit', 'HBP_pit', 'BK', 'WP', 'BF',
       'ERA+', 'FIP', 'WHIP', 'H9', 'HR9', 'BB9', 'SO9', 'SO/W', 'Throws',
       'Tm_fld', 'Lg', 'G', 'GS_fld', 'CG_fld', 'Inn', 'Ch', 'PO', 'A', 'E',
       'DP', 'Fld%', 'Rtot', 'Rtot/yr', 'RF/9', 'RF/G', 'Pos_Summary_fld',
       'Tm_cat', 'Lg_cat', 'G_cat', 'GS_cat', 'CG_cat', 'Inn_cat', 'Ch_cat',
       'PO_cat', 'A_cat', 'E_cat', 'DP_cat', 'Fld%_cat', 'Rtot_cat',
       'Rtot/yr_cat', 'Rctch', 'RF/9_cat', 'RF/G_cat', 'PB', 'WP_cat',
       'SB_cat', 'CS_cat', 'CS%', 'G_app', 'GS_app', 'Batting', 'Defense', 'P',
       'C', '1B', '2B_a

In [404]:
players["Primary_Pos_fld"].value_counts()

P     455
OF    226
C      93
1B     78
2B     75
SS     72
3B     58
Name: Primary_Pos_fld, dtype: int64

In [405]:
players["Primary_Pos_fld"].isnull().sum()

10

In [406]:
players.groupby("Primary_Pos_fld")["Fld%"].mean()

Primary_Pos_fld
1B    0.985859
2B    0.974507
3B    0.963414
C     0.984022
OF    0.969794
P     0.943871
SS    0.962181
Name: Fld%, dtype: float64

### Superior Rating

In [407]:
players["superior_rating"] = ""
players.loc[(players["Primary_Pos_fld"] == "P") & (players["Fld%"] >= 0.980), "superior_rating"] = "S"
players.loc[(players["Primary_Pos_fld"] == "C") & (players["Fld%"] >= 0.993), "superior_rating"] = "S"
players.loc[(players["Primary_Pos_fld"] == "1B") & (players["Fld%"] >= 0.995), "superior_rating"] = "S"
players.loc[(players["Primary_Pos_fld"] == "2B") & (players["Fld%"] >= 0.984), "superior_rating"] = "S"
players.loc[(players["Primary_Pos_fld"] == "3B") & (players["Fld%"] >= 0.971), "superior_rating"] = "S"
players.loc[(players["Primary_Pos_fld"] == "SS") & (players["Fld%"] >= 0.973), "superior_rating"] = "S"
players.loc[(players["Primary_Pos_fld"] == "OF") & (players["Fld%"] >= 0.990), "superior_rating"] = "S"

In [408]:
players["superior_rating"].value_counts()

     714
S    353
Name: superior_rating, dtype: int64

In [409]:
players.head(10)

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos_Summary_fld,Tm_cat,...,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE,bb_rate,bb_num_pit,k_rate_pit,k_val_pit,k_num_pit,hbp_rate_pit,hbp_val_pit,hbp_num_pit,WP_num,hr_rate_pit,gopher_ball,control_rate,PCN,hit_rate_pit,PPH,goph_lett_inn,bb_k_hbp,pitcher_rating,superior_rating
0,Bob Adams,25,DET,AL,15.0,24.0,24.0,2.0,6.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,5.0,0.250,0.250,0.542,0.792,105.0,13.0,0.0,0.0,0.0,0.0,0.0,H/32,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DET,AL,3.0,3.0,3.0,27.0,27.0,26.0,1.0,0.0,1.0,1.000,-1.0,-22.0,9.00,9.00,1B-C,DET,...,1.0,9.0,6.0,6.0,0.0,0.0,0.0,1.000,0.0,0.0,0.0,6.0,6.00,0.0,0.0,1.0,0.0,0%,15.0,3.0,15.0,3.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,1B,0.133333,NaN,C+,12,26.0,0,0.0,NaN,0.000,NaN,0,n,8,8.0,22,0,0.0,NaN,9,54,C+26 [n-22],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S
1,Glenn Adams,29,MIN,AL,95.0,290.0,269.0,32.0,91.0,17.0,0.0,6.0,49.0,0.0,2.0,18.0,30.0,0.338,0.376,0.468,0.844,130.0,126.0,5.0,0.0,0.0,3.0,3.0,D97H,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIN,AL,44.0,38.0,11.0,271.0,65.0,60.0,3.0,2.0,1.0,0.969,-3.0,-13.0,2.09,1.43,OF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.0,80.0,95.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,28.0,44.0,46.0,14.0,0.0,OF,0.515789,NaN,A+,2,12.0,0,0.0,NaN,0.000,NaN,2,12,4,6.0,16,0,0.0,NaN,11,52,A+12 [12-16],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,Mike Adams,28,CHC,NL,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000,0.000,0.000,0.000,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,/87H,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHC,NL,2.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,-120.0,0.00,0.00,OF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,NaN,1.0,0.0,OF,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.000,NaN,0,n,18,18.0,36,0,0.0,NaN,0,66,G [n-36],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,Willie Aikens,22,CAL,AL,42.0,101.0,91.0,5.0,18.0,4.0,0.0,0.0,6.0,1.0,2.0,10.0,23.0,0.198,0.277,0.242,0.519,46.0,22.0,1.0,0.0,0.0,0.0,2.0,H3D,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAL,AL,13.0,12.0,6.0,99.0,105.0,94.0,8.0,3.0,10.0,0.971,0.0,-2.0,9.27,7.85,1B,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,23.0,42.0,13.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,18.0,0.0,1B,0.142857,NaN,C,0,NaN,0,0.0,NaN,0.042,NaN,4,14,8,12.0,26,0,0.0,NaN,6,61,C [14-26],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,Santo Alcala,24,TOT,NL,38.0,30.0,28.0,1.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,15.0,0.071,0.071,0.179,0.250,-34.0,5.0,0.0,0.0,2.0,0.0,0.0,1,R,NL,3.0,7.0,0.30,4.83,38.0,12.0,8.0,0.0,0.0,2.0,117.1,126.0,66.0,63.0,13.0,54.0,6.0,73.0,3.0,3.0,6.0,520.0,79.0,4.39,1.534

### Arm Rating

In [410]:
players["G"].value_counts()

34.0     30
1.0      24
3.0      23
2.0      21
5.0      21
33.0     20
6.0      19
30.0     19
35.0     17
14.0     16
26.0     16
31.0     16
4.0      16
28.0     15
22.0     15
15.0     15
27.0     15
7.0      15
29.0     14
12.0     14
16.0     14
37.0     14
9.0      13
32.0     13
25.0     12
13.0     12
8.0      12
46.0     11
18.0     11
36.0     11
49.0     11
38.0     11
17.0     11
55.0     11
21.0     11
11.0     11
45.0     10
115.0    10
10.0     10
52.0     10
39.0      9
41.0      9
148.0     9
73.0      8
40.0      8
44.0      8
24.0      8
64.0      8
136.0     8
59.0      8
47.0      8
54.0      8
149.0     8
126.0     7
20.0      7
60.0      7
42.0      7
56.0      7
53.0      7
153.0     7
19.0      7
147.0     7
152.0     7
82.0      6
65.0      6
154.0     6
155.0     6
43.0      6
91.0      6
134.0     6
63.0      6
137.0     6
97.0      5
78.0      5
146.0     5
156.0     5
61.0      5
118.0     5
66.0      5
140.0     5
123.0     5
72.0      5
133.0     5
69.0

In [411]:
players["a_gp"] = players["A"] / players["G_app"]
players["a_gp"].mean()

1.301473318227037

In [412]:
players["arm_rating"] = 8
players.loc[(players["Primary_Pos_fld"] == "P") & (players["a_gp"] >= 0.7), "arm_rating"] = 9
players.loc[(players["Primary_Pos_fld"] == "C"), "arm_rating"] = 9
players.loc[(players["Primary_Pos_fld"] == "1B") & (players["a_gp"] >= 0.7), "arm_rating"] = 9
players.loc[(players["Primary_Pos_fld"] == "2B") & (players["a_gp"] >= 2.8), "arm_rating"] = 9
players.loc[(players["Primary_Pos_fld"] == "3B") & (players["a_gp"] >= 2.0), "arm_rating"] = 9
players.loc[(players["Primary_Pos_fld"] == "SS") & (players["a_gp"] >= 2.8), "arm_rating"] = 9
players.loc[(players["Primary_Pos_fld"] == "OF") & (players["a_gp"] >= 0.08), "arm_rating"] = 9

In [413]:
players["arm_rating"].value_counts()

8    698
9    369
Name: arm_rating, dtype: int64

### Range Rating

In [414]:
players["po_gp"] = players["PO"] / players["G_app"]
players["po_gp"].mean()

2.0369439884966742

In [415]:
players.groupby("Primary_Pos_fld")["po_gp"].mean()

Primary_Pos_fld
1B    5.904680
2B    4.349857
3B    1.034811
C     5.015489
OF    2.191354
P     0.334255
SS    2.325375
Name: po_gp, dtype: float64

In [416]:
players["range_rating"] = 4
players.loc[(players["Primary_Pos_fld"] == "P") & (players["po_gp"] >= 0.3), "range_rating"] = 5
players.loc[(players["Primary_Pos_fld"] == "C"), "range_rating"] = 4
players.loc[(players["Primary_Pos_fld"] == "1B") & (players["po_gp"] >= 8.3), "range_rating"] = 5
players.loc[(players["Primary_Pos_fld"] == "2B") & (players["po_gp"] >= 2.1), "range_rating"] = 5
players.loc[(players["Primary_Pos_fld"] == "3B") & (players["po_gp"] >= 0.8), "range_rating"] = 5
players.loc[(players["Primary_Pos_fld"] == "SS") & (players["po_gp"] >= 1.6), "range_rating"] = 5
players.loc[(players["Primary_Pos_fld"] == "OF") & (players["po_gp"] >= 2.1), "range_rating"] = 5

In [417]:
players["range_rating"].value_counts()

4    785
5    282
Name: range_rating, dtype: int64

### Catcher Caught Stealing Rate

In [418]:
players.columns

Index(['Name', 'Age', 'Tm', 'Lg_bat', 'G_bat', 'PA', 'AB', 'R_bat', 'H_bat',
       '2B', '3B', 'HR_bat', 'RBI', 'SB', 'CS', 'BB_bat', 'SO_bat', 'BA',
       'OBP', 'SLG', 'OPS', 'OPS+', 'TB', 'GDP', 'HBP_bat', 'SH', 'SF',
       'IBB_bat', 'Pos_Summary', 'Bats', 'Lg_pit', 'W', 'L', 'W-L%', 'ERA',
       'G_pit', 'GS', 'GF', 'CG', 'SHO', 'SV', 'IP', 'H_pit', 'R_pit', 'ER',
       'HR_pit', 'BB_pit', 'IBB_pit', 'SO_pit', 'HBP_pit', 'BK', 'WP', 'BF',
       'ERA+', 'FIP', 'WHIP', 'H9', 'HR9', 'BB9', 'SO9', 'SO/W', 'Throws',
       'Tm_fld', 'Lg', 'G', 'GS_fld', 'CG_fld', 'Inn', 'Ch', 'PO', 'A', 'E',
       'DP', 'Fld%', 'Rtot', 'Rtot/yr', 'RF/9', 'RF/G', 'Pos_Summary_fld',
       'Tm_cat', 'Lg_cat', 'G_cat', 'GS_cat', 'CG_cat', 'Inn_cat', 'Ch_cat',
       'PO_cat', 'A_cat', 'E_cat', 'DP_cat', 'Fld%_cat', 'Rtot_cat',
       'Rtot/yr_cat', 'Rctch', 'RF/9_cat', 'RF/G_cat', 'PB', 'WP_cat',
       'SB_cat', 'CS_cat', 'CS%', 'G_app', 'GS_app', 'Batting', 'Defense', 'P',
       'C', '1B', '2B_a

In [419]:
players["CS%"].value_counts()

0%     12
34%     6
44%     5
33%     5
36%     4
39%     4
43%     4
28%     4
25%     4
38%     4
41%     3
35%     3
83%     3
42%     3
46%     3
50%     3
17%     3
31%     3
24%     2
40%     2
11%     2
30%     2
29%     2
58%     1
18%     1
56%     1
48%     1
71%     1
10%     1
57%     1
13%     1
22%     1
37%     1
52%     1
45%     1
Name: CS%, dtype: int64

In [420]:
players["cs_rate"] = players["CS_cat"] / (players["SB_cat"] + players["CS_cat"])
players["cs_rate"].mean()

0.32800761827683617

In [421]:
cs_break_points = [
    0.21,
    0.31,
    0.41,
    0.51
]

rating = [
    "",
    "-1",
    "-2",
    "-3",
    "-4"
]

def cs_rating(cs_rate, breakpoints=cs_break_points, ratings=rating):
    i = bisect(breakpoints, cs_rate)
    return ratings[i]

In [422]:
players["cs_num"] = [cs_rating(rate) for rate in players["cs_rate"]]
players["cs_num"].value_counts()

-4    977
-2     32
-3     22
       20
-1     16
Name: cs_num, dtype: int64

In [423]:
players["G_cat"].isnull().sum()

961

In [424]:
players.loc[(players["G_cat"].isnull()), "cs_num"] = ""
players.loc[(players["SB_cat"] == 0), "cs_num"] = ""
players["cs_num"].value_counts()

      989
-2     32
-3     22
-1     16
-4      8
Name: cs_num, dtype: int64

In [425]:
players[players["cs_num"] == "-4"]

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos_Summary_fld,Tm_cat,...,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE,bb_rate,bb_num_pit,k_rate_pit,k_val_pit,k_num_pit,hbp_rate_pit,hbp_val_pit,hbp_num_pit,WP_num,hr_rate_pit,gopher_ball,control_rate,PCN,hit_rate_pit,PPH,goph_lett_inn,bb_k_hbp,pitcher_rating,superior_rating,a_gp,arm_rating,po_gp,range_rating,cs_rate,cs_num
180,Bob Davis,25,SDP,NL,48.0,104.0,94.0,9.0,17.0,2.0,0.0,1.0,10.0,0.0,0.0,5.0,24.0,0.181,0.235,0.234,0.469,33.0,22.0,2.0,2.0,2.0,1.0,2.0,2/H,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SDP,NL,46.0,31.0,15.0,263.1,159.0,136.0,19.0,4.0,2.0,0.975,4.0,18.0,5.30,3.37,C,SDP,...,2.0,0.975,4.0,18.0,4.0,5.30,3.37,0.0,8.0,12.0,16.0,57%,48.0,31.0,48.0,45.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2.0,2.0,C,0.208333,NaN,D+,2,12.0,0,0.0,NaN,0.000,NaN,2,12,8,10.0,24,1,11.0,/25,6,61,D+12 [12-24/25],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,0.395833,9,2.833333,4,0.571429,-4
187,Rick Dempsey,27,BAL,AL,91.0,314.0,270.0,27.0,61.0,7.0,4.0,3.0,34.0,2.0,3.0,34.0,34.0,0.226,0.314,0.315,0.629,77.0,85.0,9.0,2.0,5.0,3.0,1.0,2/H,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BAL,AL,91.0,84.0,74.0,745.0,479.0,416.0,52.0,11.0,10.0,0.977,9.0,14.0,5.65,5.14,C,BAL,...,10.0,0.977,9.0,14.0,9.0,5.65,5.14,2.0,15.0,30.0,41.0,58%,91.0,84.0,91.0,91.0,0.0,91.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,C,0.373626,NaN,C+,2,12.0,2,4.0,(14),0.024,NaN,4,14,4,8.0,22,0,0.0,NaN,7,56,C+12(14) [14-22],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,0.571429,9,4.571429,4,0.577465,-4
379,Cliff Johnson,29,TOT,MLB,107.0,339.0,286.0,46.0,85.0,16.0,0.0,22.0,54.0,0.0,2.0,43.0,53.0,0.297,0.407,0.584,0.991,171.0,167.0,3.0,10.0,0.0,0.0,2.0,7HD32/9,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TOT,MLB,70.0,59.0,34.0,503.0,287.0,258.0,25.0,4.0,19.0,0.986,-1.0,-3.0,5.06,4.04,OF-1B-C,NYY,...,0.0,1.000,1.0,12.0,1.0,5.64,4.13,1.0,3.0,1.0,5.0,83%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OF,0.504673,NaN,B+,9,23.0,0,0.0,NaN,0.000,NaN,5,15,6,11.0,25,1,12.0,/26,9,54,B+23 [15-25/26],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,8,NaN,4,0.833333,-4
380,Cliff Johnson,29,HOU,NL,51.0,171.0,144.0,22.0,43.0,8.0,0.0,10.0,23.0,0.0,1.0,23.0,30.0,0.299,0.409,0.563,0.972,169.0,81.0,1.0,4.0,0.0,0.0,2.0,7H3/9,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TOT,MLB,70.0,59.0,34.0,503.0,287.0,258.0,25.0,4.0,19.0,0.986,-1.0,-3.0,5.06,4.04,OF-1B-C,NYY,...,0.0,1.000,1.0,12.0,1.0,5.64,4.13,1.0,3.0,1.0,5.0,83%,51.0,38.0,51.0,40.0,0.0,0.0,10.0,0.0,0.0,0.0,33.0,0.0,4.0,34.0,NaN,12.0,0.0,OF,0.450980,NaN,B+,8,22.0,0,0.0,NaN,0.000,NaN,5,15,6,11.0,25,1,12.0,/26,9,54,B+22 [15-25/26],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,0.490196,9,5.058824,5,0.833333,-4
381,Cliff Johnson,29,NYY,AL,56.0,168.0,142.0,24.0,42.0,8.0,0.0,12.0,31.0,0.0,1.0,20.0,23.0,0.296,0.405,

### Fielder Rating

In [426]:
players["fielder_rating"] = (players["superior_rating"] + 
                             players["arm_rating"].astype(str) +
                             players["range_rating"].astype(str) + 
                             " " + 
                             players["cs_num"]
)
players["fielder_rating"].value_counts()

84        380
S84       215
95        128
94         77
85         65
S95        51
S94        37
S85        36
94 -2      28
94 -3      17
94 -1      12
S94 -2      4
S94 -1      4
S94 -3      4
94 -4       3
95 -4       2
S94 -4      2
84 -4       1
84 -3       1
Name: fielder_rating, dtype: int64

In [427]:
players[players["fielder_rating"] == "95 -2"]

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos_Summary_fld,Tm_cat,...,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE,bb_rate,bb_num_pit,k_rate_pit,k_val_pit,k_num_pit,hbp_rate_pit,hbp_val_pit,hbp_num_pit,WP_num,hr_rate_pit,gopher_ball,control_rate,PCN,hit_rate_pit,PPH,goph_lett_inn,bb_k_hbp,pitcher_rating,superior_rating,a_gp,arm_rating,po_gp,range_rating,cs_rate,cs_num,fielder_rating


In [428]:
players.to_csv("../data/player stats - " + year + " - with batter pitcher and fielder ratings.csv", index=False)

# Save teams to separate Excel tabs

In [429]:
players = pd.read_csv("../data/player stats - " + year + " - with batter pitcher and fielder ratings.csv")

In [430]:
players.head()

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos_Summary_fld,Tm_cat,...,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE,bb_rate,bb_num_pit,k_rate_pit,k_val_pit,k_num_pit,hbp_rate_pit,hbp_val_pit,hbp_num_pit,WP_num,hr_rate_pit,gopher_ball,control_rate,PCN,hit_rate_pit,PPH,goph_lett_inn,bb_k_hbp,pitcher_rating,superior_rating,a_gp,arm_rating,po_gp,range_rating,cs_rate,cs_num,fielder_rating
0,Bob Adams,25,DET,AL,15.0,24.0,24.0,2.0,6.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,5.0,0.250,0.250,0.542,0.792,105.0,13.0,0.0,0.0,0.0,0.0,0.0,H/32,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DET,AL,3.0,3.0,3.0,27.0,27.0,26.0,1.0,0.0,1.0,1.000,-1.0,-22.0,9.00,9.00,1B-C,DET,...,1.0,0.0,0.0,0.0,6.0,6.0,0.0,0.0,1.0,0.0,0%,15.0,3.0,15.0,3.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,1B,0.133333,NaN,C+,12,26.0,0,0.0,NaN,0.000,NaN,0,n,8,8.0,22,0,0.0,NaN,9,54,C+26 [n-22],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,0.066667,8,1.733333,4,0.0,NaN,S84
1,Glenn Adams,29,MIN,AL,95.0,290.0,269.0,32.0,91.0,17.0,0.0,6.0,49.0,0.0,2.0,18.0,30.0,0.338,0.376,0.468,0.844,130.0,126.0,5.0,0.0,0.0,3.0,3.0,D97H,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIN,AL,44.0,38.0,11.0,271.0,65.0,60.0,3.0,2.0,1.0,0.969,-3.0,-13.0,2.09,1.43,OF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.0,80.0,95.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,28.0,44.0,46.0,14.0,0.0,OF,0.515789,NaN,A+,2,12.0,0,0.0,NaN,0.000,NaN,2,12,4,6.0,16,0,0.0,NaN,11,52,A+12 [12-16],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.031579,8,0.631579,4,NaN,NaN,84
2,Mike Adams,28,CHC,NL,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000,0.000,0.000,0.000,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,/87H,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHC,NL,2.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,-120.0,0.00,0.00,OF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,NaN,1.0,0.0,OF,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.000,NaN,0,n,18,18.0,36,0,0.0,NaN,0,66,G [n-36],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,8,0.000000,4,NaN,NaN,84
3,Willie Aikens,22,CAL,AL,42.0,101.0,91.0,5.0,18.0,4.0,0.0,0.0,6.0,1.0,2.0,10.0,23.0,0.198,0.277,0.242,0.519,46.0,22.0,1.0,0.0,0.0,0.0,2.0,H3D,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAL,AL,13.0,12.0,6.0,99.0,105.0,94.0,8.0,3.0,10.0,0.971,0.0,-2.0,9.27,7.85,1B,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,23.0,42.0,13.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,18.0,0.0,1B,0.142857,NaN,C,0,NaN,0,0.0,NaN,0.042,NaN,4,14,8,12.0,26,0,0.0,NaN,6,61,C [14-26],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.190476,8,2.238095,4,NaN,NaN,84
4,Santo Alcala,24,TOT,NL,38.0,30.0,28.0,1.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,15.0,0.071,0.071,0.179,0.250,-34.0,5.0,0.0,0.0,2.0,0.0,0.0,1,R,NL,3.0,7.0,0.3,4.83,38.0,12.0,8.0,0.0,0.0,2.0,117.1,126.0,66.0,63.0,13.0,54.0

In [431]:
pd.set_option('display.max_seq_items', 175)
players.columns

Index(['Name', 'Age', 'Tm', 'Lg_bat', 'G_bat', 'PA', 'AB', 'R_bat', 'H_bat',
       '2B', '3B', 'HR_bat', 'RBI', 'SB', 'CS', 'BB_bat', 'SO_bat', 'BA',
       'OBP', 'SLG', 'OPS', 'OPS+', 'TB', 'GDP', 'HBP_bat', 'SH', 'SF',
       'IBB_bat', 'Pos_Summary', 'Bats', 'Lg_pit', 'W', 'L', 'W-L%', 'ERA',
       'G_pit', 'GS', 'GF', 'CG', 'SHO', 'SV', 'IP', 'H_pit', 'R_pit', 'ER',
       'HR_pit', 'BB_pit', 'IBB_pit', 'SO_pit', 'HBP_pit', 'BK', 'WP', 'BF',
       'ERA+', 'FIP', 'WHIP', 'H9', 'HR9', 'BB9', 'SO9', 'SO/W', 'Throws',
       'Tm_fld', 'Lg', 'G', 'GS_fld', 'CG_fld', 'Inn', 'Ch', 'PO', 'A', 'E',
       'DP', 'Fld%', 'Rtot', 'Rtot/yr', 'RF/9', 'RF/G', 'Pos_Summary_fld',
       'Tm_cat', 'Lg_cat', 'G_cat', 'GS_cat', 'CG_cat', 'Inn_cat', 'Ch_cat',
       'PO_cat', 'A_cat', 'E_cat', 'DP_cat', 'Fld%_cat', 'Rtot_cat',
       'Rtot/yr_cat', 'Rctch', 'RF/9_cat', 'RF/G_cat', 'PB', 'WP_cat',
       'SB_cat', 'CS_cat', 'CS%', 'G_app', 'GS_app', 'Batting', 'Defense', 'P',
       'C', '1B', '2B_a

In [432]:
pd.set_option('display.max_columns', 175)
players.head()

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos_Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE,bb_rate,bb_num_pit,k_rate_pit,k_val_pit,k_num_pit,hbp_rate_pit,hbp_val_pit,hbp_num_pit,WP_num,hr_rate_pit,gopher_ball,control_rate,PCN,hit_rate_pit,PPH,goph_lett_inn,bb_k_hbp,pitcher_rating,superior_rating,a_gp,arm_rating,po_gp,range_rating,cs_rate,cs_num,fielder_rating
0,Bob Adams,25,DET,AL,15.0,24.0,24.0,2.0,6.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,5.0,0.250,0.250,0.542,0.792,105.0,13.0,0.0,0.0,0.0,0.0,0.0,H/32,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DET,AL,3.0,3.0,3.0,27.0,27.0,26.0,1.0,0.0,1.0,1.000,-1.0,-22.0,9.00,9.00,1B-C,DET,AL,1.0,1.0,1.0,9.0,6.0,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0,6.0,0.0,0.0,1.0,0.0,0%,15.0,3.0,15.0,3.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,1B,0.133333,NaN,C+,12,26.0,0,0.0,NaN,0.000,NaN,0,n,8,8.0,22,0,0.0,NaN,9,54,C+26 [n-22],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,0.066667,8,1.733333,4,0.0,NaN,S84
1,Glenn Adams,29,MIN,AL,95.0,290.0,269.0,32.0,91.0,17.0,0.0,6.0,49.0,0.0,2.0,18.0,30.0,0.338,0.376,0.468,0.844,130.0,126.0,5.0,0.0,0.0,3.0,3.0,D97H,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIN,AL,44.0,38.0,11.0,271.0,65.0,60.0,3.0,2.0,1.0,0.969,-3.0,-13.0,2.09,1.43,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.0,80.0,95.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,28.0,44.0,46.0,14.0,0.0,OF,0.515789,NaN,A+,2,12.0,0,0.0,NaN,0.000,NaN,2,12,4,6.0,16,0,0.0,NaN,11,52,A+12 [12-16],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.031579,8,0.631579,4,NaN,NaN,84
2,Mike Adams,28,CHC,NL,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000,0.000,0.000,0.000,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,/87H,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHC,NL,2.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,-120.0,0.00,0.00,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,NaN,1.0,0.0,OF,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.000,NaN,0,n,18,18.0,36,0,0.0,NaN,0,66,G [n-36],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,8,0.000000,4,NaN,NaN,84
3,Willie Aikens,22,CAL,AL,42.0,101.0,91.0,5.0,18.0,4.0,0.0,0.0,6.0,1.0,2.0,10.0,23.0,0.198,0.277,0.242,0.519,46.0,22.0,1.0,0.0,0.0,0.0,2.0,H3D,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAL,AL,13.0,12.0,6.0,99.0,105.0,94.0,8.0,3.0,10.0,0.971,0.0,-2.0,9.27,7.85,1B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,23.0,42.0,13.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,18.0,0.0,1B,0.142857,NaN,C,0,NaN,0,0.0,NaN,0.042,NaN,4,14,8,12.0,26,0,0.0,NaN,6,61,C [14-26],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.190476,8,2.238095,4,NaN,NaN,84
4,Santo A

In [433]:
if 'DH' not in players:
    players['DH'] = 0
players.head()

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos_Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE,bb_rate,bb_num_pit,k_rate_pit,k_val_pit,k_num_pit,hbp_rate_pit,hbp_val_pit,hbp_num_pit,WP_num,hr_rate_pit,gopher_ball,control_rate,PCN,hit_rate_pit,PPH,goph_lett_inn,bb_k_hbp,pitcher_rating,superior_rating,a_gp,arm_rating,po_gp,range_rating,cs_rate,cs_num,fielder_rating
0,Bob Adams,25,DET,AL,15.0,24.0,24.0,2.0,6.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,5.0,0.250,0.250,0.542,0.792,105.0,13.0,0.0,0.0,0.0,0.0,0.0,H/32,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DET,AL,3.0,3.0,3.0,27.0,27.0,26.0,1.0,0.0,1.0,1.000,-1.0,-22.0,9.00,9.00,1B-C,DET,AL,1.0,1.0,1.0,9.0,6.0,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0,6.0,0.0,0.0,1.0,0.0,0%,15.0,3.0,15.0,3.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,1B,0.133333,NaN,C+,12,26.0,0,0.0,NaN,0.000,NaN,0,n,8,8.0,22,0,0.0,NaN,9,54,C+26 [n-22],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,0.066667,8,1.733333,4,0.0,NaN,S84
1,Glenn Adams,29,MIN,AL,95.0,290.0,269.0,32.0,91.0,17.0,0.0,6.0,49.0,0.0,2.0,18.0,30.0,0.338,0.376,0.468,0.844,130.0,126.0,5.0,0.0,0.0,3.0,3.0,D97H,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIN,AL,44.0,38.0,11.0,271.0,65.0,60.0,3.0,2.0,1.0,0.969,-3.0,-13.0,2.09,1.43,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.0,80.0,95.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,28.0,44.0,46.0,14.0,0.0,OF,0.515789,NaN,A+,2,12.0,0,0.0,NaN,0.000,NaN,2,12,4,6.0,16,0,0.0,NaN,11,52,A+12 [12-16],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.031579,8,0.631579,4,NaN,NaN,84
2,Mike Adams,28,CHC,NL,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000,0.000,0.000,0.000,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,/87H,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHC,NL,2.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,-120.0,0.00,0.00,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,NaN,1.0,0.0,OF,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.000,NaN,0,n,18,18.0,36,0,0.0,NaN,0,66,G [n-36],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,8,0.000000,4,NaN,NaN,84
3,Willie Aikens,22,CAL,AL,42.0,101.0,91.0,5.0,18.0,4.0,0.0,0.0,6.0,1.0,2.0,10.0,23.0,0.198,0.277,0.242,0.519,46.0,22.0,1.0,0.0,0.0,0.0,2.0,H3D,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAL,AL,13.0,12.0,6.0,99.0,105.0,94.0,8.0,3.0,10.0,0.971,0.0,-2.0,9.27,7.85,1B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,23.0,42.0,13.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,18.0,0.0,1B,0.142857,NaN,C,0,NaN,0,0.0,NaN,0.042,NaN,4,14,8,12.0,26,0,0.0,NaN,6,61,C [14-26],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.190476,8,2.238095,4,NaN,NaN,84
4,Santo A

In [434]:
# fix games played column
players.loc[(players["Primary_Pos_fld"] == "P"), "Games_Played"] = players["G_pit"]
players.loc[(players["Primary_Pos_fld"] != "P"), "Games_Played"] = players["G_bat"]
players["Games_Played"].value_counts()

3.0      24
13.0     23
2.0      22
1.0      22
4.0      22
5.0      20
6.0      20
34.0     19
8.0      17
7.0      17
12.0     17
33.0     17
11.0     16
14.0     16
154.0    14
17.0     14
36.0     14
29.0     13
41.0     13
10.0     13
32.0     13
15.0     13
49.0     13
25.0     13
31.0     13
22.0     13
16.0     12
30.0     12
39.0     12
42.0     12
37.0     11
26.0     11
56.0     11
9.0      11
153.0    10
18.0     10
150.0    10
35.0     10
27.0     10
64.0      9
20.0      9
21.0      9
19.0      9
152.0     9
59.0      8
139.0     8
60.0      8
38.0      8
66.0      8
51.0      8
24.0      8
23.0      7
40.0      7
43.0      7
58.0      7
86.0      7
53.0      7
144.0     6
148.0     6
72.0      6
158.0     6
55.0      6
118.0     6
52.0      6
54.0      6
44.0      6
69.0      6
65.0      6
159.0     6
129.0     6
80.0      5
157.0     5
46.0      5
61.0      5
45.0      5
96.0      5
149.0     5
141.0     5
73.0      5
28.0      5
151.0     5
120.0     5
94.0      5
78.0

In [435]:
players_short = players.loc[:, ["Name", "Age", "Tm", "Games_Played", "GS", "GF", "Pos_Summary_fld", 
                                "fielder_rating", "batter_rating", "PH_num_bat", "Bats", "pitcher_rating", "PCN", 
                                "PPH", "Throws", "Primary_Pos_fld", "P", "C", "1B", "2B_app", "3B_app", "SS", 
                                "LF", "CF", "RF", "OF", "DH", "PH", "PR"]]
players_short.rename(columns={
    "Games_Played": "G",
    "Pos_Summary_fld": "Positions",
    "fielder_rating": "DEF",
    "batter_rating": "Batter Rating",
    "PH_num_bat": "BPH",
    "Bats": "B",
    "pitcher_rating": "Pitcher Rating",
    "Throws": "T",
    "Primary_Pos_fld": "Primary",
    "2B_app": "2B",
    "3B_app": "3B"
}, inplace=True)
players_short.head()

,Name,Age,Tm,G,GS,GF,Positions,DEF,Batter Rating,BPH,B,Pitcher Rating,PCN,PPH,T,Primary,P,C,1B,2B,3B,SS,LF,CF,RF,OF,DH,PH,PR
0,Bob Adams,25,DET,15.0,NaN,NaN,1B-C,S84,C+26 [n-22],54,R,NaN,NaN,NaN,NaN,1B,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0
1,Glenn Adams,29,MIN,95.0,NaN,NaN,OF,84,A+12 [12-16],52,L,NaN,NaN,NaN,NaN,OF,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,28.0,44.0,46.0,14.0,0.0
2,Mike Adams,28,CHC,2.0,NaN,NaN,OF,84,G [n-36],66,R,NaN,NaN,NaN,NaN,OF,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,NaN,1.0,0.0
3,Willie Aikens,22,CAL,42.0,NaN,NaN,1B,84,C [14-26],61,L,NaN,NaN,NaN,NaN,1B,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,18.0,0.0
4,Santo Alcala,24,TOT,38.0,12.0,8.0,P,S84,F36 [n-36],65,R,+W3 (14-23) [WP],45.0,54.0,R,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [436]:
players_short["Primary"].replace(np.nan, "DH_PH_PR", inplace=True)

In [437]:
pos_cat_dtype = pd.api.types.CategoricalDtype(categories=["C", "1B", "2B", "3B", "SS", "OF", "DH_PH_PR", "P", ""],
                                              ordered=True) 
players_short["Primary"] = players_short['Primary'].astype(pos_cat_dtype)
players_short["Primary"].value_counts()

P           455
OF          226
C            93
1B           78
2B           75
SS           72
3B           58
DH_PH_PR     10
              0
Name: Primary, dtype: int64

In [438]:
players_short.fillna("", inplace=True)
players_short.head()

,Name,Age,Tm,G,GS,GF,Positions,DEF,Batter Rating,BPH,B,Pitcher Rating,PCN,PPH,T,Primary,P,C,1B,2B,3B,SS,LF,CF,RF,OF,DH,PH,PR
0,Bob Adams,25,DET,15.0,,,1B-C,S84,C+26 [n-22],54,R,,,,,1B,0,1,2,0,0,0,0,0,0,0,0,12,0
1,Glenn Adams,29,MIN,95.0,,,OF,84,A+12 [12-16],52,L,,,,,OF,0,0,0,0,0,0,16,0,28,44,46,14,0
2,Mike Adams,28,CHC,2.0,,,OF,84,G [n-36],66,R,,,,,OF,0,0,0,0,0,0,1,1,0,2,,1,0
3,Willie Aikens,22,CAL,42.0,,,1B,84,C [14-26],61,L,,,,,1B,0,0,13,0,0,0,0,0,0,0,13,18,0
4,Santo Alcala,24,TOT,38.0,12,8,P,S84,F36 [n-36],65,R,+W3 (14-23) [WP],45,54,R,P,,,,,,,,,,,,,


In [439]:
players_short["Name"] = players_short["Name"].str.replace("\xa0", " ")

In [440]:
# players_short[players_short["Name"] == "Steve Carlton"]

In [441]:
players_short.drop_duplicates(["Name", "Tm"], keep='first', inplace=True)

In [442]:
# players_short[players_short["Name"] == "Steve Carlton"]

In [443]:
players_short = players_short.sort_values(["Tm", "Primary", "G", "GS", "GF"], 
                                          ascending = (True, True, False, False, False))
players_short.head(50)

,Name,Age,Tm,G,GS,GF,Positions,DEF,Batter Rating,BPH,B,Pitcher Rating,PCN,PPH,T,Primary,P,C,1B,2B,3B,SS,LF,CF,RF,OF,DH,PH,PR
622,Biff Pocoroba,23,ATL,113.0,,,C,94 -2,B+13 [15-22],54,S,,,,,C,0,100,0,0,0,0,0,0,0,0,,23,0
574,Joe Nolan,26,ATL,62.0,,,C,S94,B+15 [15-23],54,L,,,,,C,0,19,0,0,0,0,0,0,0,0,,48,0
152,Vic Correll,31,ATL,54.0,,,C,94 -1,C22 [15-26],61,R,,,,,C,0,49,0,0,0,0,0,0,0,0,,7,0
559,Dale Murphy,21,ATL,18.0,,,C,94,#A13(15) [n-14],52,R,,,,,C,0,18,0,0,0,0,0,0,0,0,,0,0
541,Willie Montanez,29,ATL,136.0,,,1B,85,B+15 [12-16],53,L,,,,,1B,0,0,134,0,0,0,0,0,0,0,,2,0
598,Tom Paciorek,30,ATL,72.0,,,1B-OF-3B,84,C+13 [11-25],55,R,,,,,1B,0,0,32,0,1,0,4,2,3,9,,35,0
271,Rod Gilbreath,24,ATL,128.0,,,2B-3B,85,C+13(14) [13-23],55,R,,,,,2B,0,0,0,122,1,0,0,0,0,0,,6,1
676,Jerry Royster,24,ATL,140.0,,,3B-SS-2B-OF,85,C12(13)*** [13-22],56,R,,,,,3B,0,0,0,39,55,52,0,1,0,1,,6,11
546,Junior Moore,24,ATL,112.0,,,3B-2B,84,B12(13) [13-16],55,R,,,,,3B,0,0,0,1,104,0,0,0,0,0,,14,2
665,Pat Rockett,22,ATL,93.0,,,SS,85,B11 [13-21],55,R,,,,,SS,0,0,0,0,0,84,0,0,0,0,,3,8


In [444]:
my_dict = {index: group_teams for index, group_teams in players_short.groupby('Tm')}
my_dict

{'ATL':                  Name  Age   Tm      G  GS  GF    Positions    DEF  \
 622     Biff Pocoroba   23  ATL  113.0                    C  94 -2   
 574         Joe Nolan   26  ATL   62.0                    C   S94    
 152       Vic Correll   31  ATL   54.0                    C  94 -1   
 559       Dale Murphy   21  ATL   18.0                    C    94    
 541   Willie Montanez   29  ATL  136.0                   1B    85    
 598      Tom Paciorek   30  ATL   72.0             1B-OF-3B    84    
 271     Rod Gilbreath   24  ATL  128.0                2B-3B    85    
 676     Jerry Royster   24  ATL  140.0          3B-SS-2B-OF    85    
 546      Junior Moore   24  ATL  112.0                3B-2B    84    
 665       Pat Rockett   22  ATL   93.0                   SS    85    
 137     Darrel Chaney   29  ATL   74.0                SS-2B   S84    
 664    Craig Robinson   28  ATL   27.0                   SS   S84    
 52        Rob Belloir   28  ATL    6.0                   SS   S84    

In [445]:
# https://stackoverflow.com/questions/21981820/creating-multiple-excel-worksheets-using-data-in-a-pandas-dataframe/21984437
writer = pd.ExcelWriter('../data/' + year + ' rosters ' + '.xlsx', engine='xlsxwriter')

def create_excel(dictionary):
    count = 0
    for i, j in dictionary.items():
        dictionary[i].to_excel(writer, sheet_name=i)
        count += 1
    
    writer.save()
    return count

In [446]:
create_excel(my_dict)

27

## Format Excel file

In [447]:
wb = openpyxl.load_workbook('../data/' + year + ' rosters ' + '.xlsx')

In [448]:
# wb.sheetnames

In [449]:
team_dict = {
    "ANA": "Anaheim Angels",
    "ARI": "Arizona Diamondbacks",
    "ATL": "Atlanta Braves",
    "BAL": "Baltimore Orioles",
    "BOS": "Boston Red Sox",
    "BRO": "Brooklyn Dodgers",
    "CAL": "California Angels",
    "CHC": "Chicago Cubs",
    "CHW": "Chicago White Sox",
    "CIN": "Cincinnati Reds",
    "CLE": "Cleveland Indians",
    "COL": "Colorado Rockies",
    "DET": "Detroit Tigers",
    "FLA": "Florida Marlins",
    "HOU": "Houston Astros",
    "KCA": "Kansas City Athletics",
    "KCR": "Kansas City Royals",
    "LAA": "Los Angeles Angels",
    "LAD": "Los Angeles Dodgers",
    "MIA": "Miami Marlins",
    "MLN": "Milwaukee Braves",
    "MIL": "Milwaukee Brewers",
    "MIN": "Minnesota Twins",
    "MON": "Montreal Expos",
    "NYG": "New York Giants",
    "NYM": "New York Mets",
    "NYY": "New York Yankees",
    "OAK": "Oakland A's",
    "PHI": "Philadelphia Phillies",
    "PIT": "Pittsburgh Pirates",
    "SDP": "San Diego Padres",
    "SEA": "Seattle Mariners",
    "SEP": "Seattle Pilots",
    "SFG": "San Francisco Giants",
    "STL": "St. Louis Cardinals",
    "TBD": "Tampa Bay Devil Rays",
    "TBR": "Tampa Bay Rays",
    "TEX": "Texas Rangers",
    "TOR": "Toronto Blue Jays",
    "TOT": "Muli-team Totals",
    "WSN": "Washington Nationals",
    "WSA": "Washington Senators",
    "WSH": "Washington Senators"
}

In [450]:
# team_dict["ATL"]

In [451]:
header_fill = openpyxl.styles.colors.Color(rgb='00FFFFFF')
style = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
                       showLastColumn=False, showRowStripes=True, showColumnStripes=False)
border = Border(left=Side(border_style='thin', color='FF000000'),
                right=Side(border_style='thin', color='FF000000'),
                top=Side(border_style='thin', color='FF000000'),
                bottom=Side(border_style='thin', color='FF000000')
)
alignment = Alignment(horizontal='center')
width_1 = 3
width_2 = 4
width_3 = 5
width_4 = 6
width_5 = 8
width_6 = 12
width_7 = 14
width_8 = 22
width_9 = 24

for sheet in wb:

    sheetname = sheet.title
    sheet.insert_rows(1)
    row_count = sheet.max_row
    column_count = sheet.max_column
    max_cell = "A2:" + str(get_column_letter(column_count)) + str(row_count) + ""
    all_cells = "A1:" + str(get_column_letter(column_count)) + str(row_count) + ""

    sheet.merge_cells('A1:AD1')
    sheet['A1'].alignment = Alignment(horizontal='center')
    sheet['A1'].fill = PatternFill(patternType='solid', fgColor=header_fill)
    sheet['A1'].value = year + ' ' + team_dict[sheetname]
    sheet['A2'].value = "ID"
    tab = Table(displayName="Table" + sheetname, ref=max_cell)
    tab.tableStyleInfo = style
    sheet.add_table(tab)
    sheet.column_dimensions['A'].width = width_4    
    sheet.column_dimensions['B'].width = width_8
    sheet.column_dimensions['C'].width = width_2
    sheet.column_dimensions['D'].width = width_3
    sheet.column_dimensions['E'].width = width_3
    sheet.column_dimensions['F'].width = width_2
    sheet.column_dimensions['G'].width = width_2
    sheet.column_dimensions['H'].width = width_7
    sheet.column_dimensions['I'].width = width_5
    sheet.column_dimensions['J'].width = width_9
    sheet.column_dimensions['K'].width = width_3
    sheet.column_dimensions['L'].width = width_1
    sheet.column_dimensions['M'].width = width_9
    sheet.column_dimensions['N'].width = width_3
    sheet.column_dimensions['O'].width = width_3
    sheet.column_dimensions['P'].width = width_1
    sheet.column_dimensions['Q'].width = width_6
    sheet.column_dimensions['R'].width = width_3
    sheet.column_dimensions['S'].width = width_3
    sheet.column_dimensions['T'].width = width_3
    sheet.column_dimensions['U'].width = width_3
    sheet.column_dimensions['V'].width = width_3
    sheet.column_dimensions['W'].width = width_3
    sheet.column_dimensions['X'].width = width_3
    sheet.column_dimensions['Y'].width = width_3
    sheet.column_dimensions['Z'].width = width_3
    sheet.column_dimensions['AA'].width = width_3
    sheet.column_dimensions['AB'].width = width_3
    sheet.column_dimensions['AC'].width = width_3
    sheet.column_dimensions['AD'].width = width_3
    
    rows = sheet[max_cell]
    for row in rows:
        for cell in row:
            cell.border = border
            cell.alignment = alignment
            cell.font = Font(size = 14)
    
    sheet['A1'].font = Font(size = 32, bold = True, color='005A80B8')
    sheet['A2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['B2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['C2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['D2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['E2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['F2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['G2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['H2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['I2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['J2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['K2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['L2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['M2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['N2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['O2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['P2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['Q2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['R2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['S2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['T2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['U2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['V2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['W2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['X2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['Y2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['Z2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['AA2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['AB2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['AC2'].font = Font(size = 14, bold = True, color='00FFFFFF')
    sheet['AD2'].font = Font(size = 14, bold = True, color='00FFFFFF')
            

wb.save('../data/' + year + ' rosters ' + 'formatted.xlsx')

# Clean up

## Remove unwanted files

In [452]:
os.remove('../data/' + year + ' rosters .xlsx')
os.remove('../data/player stats - ' + year + ' - with batter and pitcher ratings.csv')
os.remove('../data/player stats - ' + year + ' - with batter pitcher and fielder ratings.csv')
os.remove('../data/player stats - ' + year + ' - with batter ratings.csv')
os.remove('../data/player stats - ' + year + '.csv')